In [ ]:
aliencat = False
dgx = True


batch_mode = True
remote_dump = True

negated_presence_annotations = True
store_annot = True

share_sftp = True
multi_process = True

annot_first = True
strip_list = True

In [ ]:
import time

# time.sleep(86000 * 4)

#%%javascript
#IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
# %pip install 'paramiko-3.1.0-py3-none-any.whl'

In [ ]:
import os
from pathlib import Path
import paramiko
from os.path import exists
import random
from datetime import datetime, timezone

# nb_full_path = os.path.join(os.getcwd(), nb_name)

In [ ]:
nb_name = 3.5

In [ ]:
if(not aliencat or dgx):

    import sys
    import logging

    nblog = open(f"{nb_name}.log", "w")
    nblog = open(f"{nb_name}.log", "a+")
    sys.stdout.echo = nblog
    sys.stderr.echo = nblog

    get_ipython().log.handlers[0].stream = nblog
    get_ipython().log.setLevel(logging.INFO)

    get_ipython().run_line_magic('autosave', '5')

In [ ]:
# from tqdm import trange
from colorama import Fore, Style

color_bars = [
    Fore.RED,
    Fore.GREEN,
    Fore.BLUE,
    Fore.MAGENTA,
    Fore.YELLOW,
    Fore.CYAN,
    Fore.WHITE,
]

In [ ]:
from IPython.utils import io

In [ ]:
import pandas as pd
import numpy as np
from multiprocessing import Pool

# import tqdm
import sys
import sys

sys.path.insert(0, "/home/aliencat/samora/gloabl_files")
sys.path.insert(0, "/data/AS/Samora/gloabl_files")
# from cogstack_v8 import *
from cogstack_v8_lite import *
from credentials import *

# import cogstack_v8
from COGStats import *
from scipy import stats
import pickle

In [ ]:
import traceback


def convert_date(date_string):
    date_string = date_string.split("T")[0]
    date_object = datetime.strptime(date_string, "%Y-%m-%d")
    return date_object

In [ ]:
def get_free_gpu():
    gpu_stats = subprocess.check_output(
        ["nvidia-smi", "--format=csv", "--query-gpu=memory.used,memory.free"]
    )
    gpu_df = pd.read_csv(
        StringIO(gpu_stats.decode("utf-8")),
        names=["memory.used", "memory.free"],
        skiprows=1,
    )
    print("GPU usage:\n{}".format(gpu_df))
    gpu_df["memory.free"] = gpu_df["memory.free"].map(lambda x: x.rstrip(" [MiB]"))
    idx = gpu_df["memory.free"].astype(int).idxmax()
    print(
        "Returning GPU{} with {} free MiB".format(idx, gpu_df.iloc[idx]["memory.free"])
    )
    return int(idx), gpu_df.iloc[idx]["memory.free"]


import subprocess
from io import StringIO

gpu_index, free_mem = get_free_gpu()

if int(free_mem) > 4000:
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_index)
    print(f"Setting gpu with {free_mem} free")
else:
    print(f"Setting NO gpu, most free memory: {free_mem} !")
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
from medcat.cat import CAT

In [ ]:
pwd

In [ ]:
# suffix='_time'
suffix = ""


treatment_doc_filename = "treatment_docs_v3.csv"
treatment_control_ratio_n = 1  # 1:n
pre_annotation_path = f"current_pat_annots_parts{suffix}/"
pre_annotation_path_mrc = f"current_pat_annots_mrc_parts{suffix}/"


print(pre_annotation_path)
print(pre_annotation_path_mrc)

In [ ]:

if not remote_dump:

    pre_path = "/mnt/hdd1/samora/HFE_time_v1/"

    pre_annotation_path = pre_path + pre_annotation_path

    pre_annotation_path_mrc = pre_path + pre_annotation_path_mrc

    Path(pre_annotation_path).mkdir(parents=True, exist_ok=True)
    Path(pre_annotation_path_mrc).mkdir(parents=True, exist_ok=True)

    print(pre_annotation_path)
    print(pre_annotation_path_mrc)

In [ ]:
current_pat_line_path = f"current_pat_lines_parts{suffix}/"
from pathlib import Path

if not remote_dump:

    current_pat_line_path = pre_path + current_pat_line_path

    current_pat_lines_path = current_pat_line_path

    Path(current_pat_line_path).mkdir(parents=True, exist_ok=True)


print(current_pat_line_path)

In [ ]:
# Get treatment docs if not already

In [ ]:
file_exists = exists(treatment_doc_filename)
if not file_exists:
    docs = cohort_searcher_no_terms(
        index_name="epr_documents",
        fields_list="""client_idcode document_guid	document_description	body_analysed updatetime clientvisit_visitidcode""".split(),
        search_string=""" "Haemochromatosis" 
                        OR "Hemochromatosis" 
                        OR "HFE" 
                        OR "HHC" 
                        OR "c282y" 
                        OR "h63d" 
                        OR "S65C" 
                        OR "Cys282Tyr" 
                        OR "p.C282Y" 
                        OR "HHemochromatosis" 
                        OR "HLAH"
                        OR "Bronze diabetes"
                        OR "Bronzed cirrhosis"
                        OR "282y"
                        OR "282C/Y"
                        OR "rs1799945"
                        OR "rs1800562"
                        OR "rs1800730"
                        OR "c.187C>G"
                        OR "c.845G>A"
                        OR "c.193A>T"
                        OR "p.His63Asp"
                        OR "p. Cys282Tyr"
                        OR "p.Ser65Cys"
                        OR "Transfusional haemosiderosis"
                        OR "Gly320Val"
                        OR "Troisier"
                        OR "Iron Storage Disorder"
                        OR "C282Y/H63D"
                        
                         
                         
                        """,
    )
    docs.to_csv(treatment_doc_filename)
else:
    docs = pd.read_csv(treatment_doc_filename)

print(len(docs))

In [ ]:
len(docs)

In [ ]:
len(docs["client_idcode"].unique())

In [ ]:
use_filter = False
if use_filter:
    json_filter_path = (
        "/data/AS/Samora/HFE/HFE/v18/MedCAT_Export_With_Text_2022-12-24_21_30_48.json"
    )
    import json

    with open(json_filter_path, "r") as f:
        json_data = json.load(f)

    len(json_data["projects"][0])
    json_cuis = json_data["projects"][0]["cuis"].split(",")
    cat.cdb.filter_by_cui(json_cuis)

In [ ]:
treatment_client_id_list = list(docs["client_idcode"].unique())

In [ ]:
len(treatment_client_id_list)

In [ ]:

random.seed(42)
use_controls = False
if use_controls:
    # Get control docs default 1:1

    all_idcodes = pd.read_csv("all_client_idcodes_epr_unique.csv")["client_idcode"]

    print(len(all_idcodes), len(treatment_client_id_list))

    full_control_client_id_list = list(set(all_idcodes) - set(treatment_client_id_list))

    full_control_client_id_list.sort()  # ensure sort for repeatability

    len(full_control_client_id_list) - len(all_idcodes)

    n_treatments = len(treatment_client_id_list) * treatment_control_ratio_n
    print(
        f"{n_treatments} selected as controls"
    )  # Soft control selection, many treatments will be false positives
    treatment_control_sample = pd.Series(full_control_client_id_list).sample(
        n_treatments, random_state=42
    )

    treatment_control_sample

    all_patient_list_control = list(treatment_control_sample.values)

    with open("control_list.pkl", "wb") as f:
        pickle.dump(all_patient_list_control, f)

    print(all_patient_list_control[0:10])

In [ ]:
all_patient_list = list(treatment_client_id_list)

In [ ]:
if use_controls:
    all_patient_list = all_patient_list + all_patient_list_control

In [ ]:
patient_concatted_master_list = []

In [ ]:
all_df_builder = None

In [ ]:
save_threshold = 200

failed_list = []

In [ ]:
random.shuffle(all_patient_list)

In [ ]:
print(len(all_patient_list))

In [ ]:
def exist_check(path, sftp_obj=None):
    if remote_dump:
        return sftp_exists(path, sftp_obj)
    else:
        return exists(path)

In [ ]:
print(f"remote_dump {remote_dump}")
print(pre_annotation_path)
print(pre_annotation_path_mrc)
print(current_pat_line_path)

if remote_dump:

    pre_path = "/mnt/hdd1/samora/HFE_time_v1/"

    def sftp_exists(path, sftp_obj=None):
        try:
            if not share_sftp:
                ssh_client = paramiko.SSHClient()
                ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
                ssh_client.connect(
                    hostname=hostname, username=username, password=password
                )

                sftp_obj = ssh_client.open_sftp()

            sftp_obj.stat(path)

            if not share_sftp:
                sftp_obj.close()
                sftp_obj.close()
            return True
        except FileNotFoundError:
            return False

    # Set the hostname, username, and password for the remote machine

    if not aliencat or dgx:
        hostname = "%HOSTIPADDRESS%"

    if aliencat and not dgx:
        hostname = "localhost"

    username = "%USERNAME%"
    password = "%PASSWORD%"

    # Create an SSH client and connect to the remote machine
    ssh_client = paramiko.SSHClient()
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh_client.connect(hostname=hostname, username=username, password=password)

    sftp_client = ssh_client.open_sftp()

    if remote_dump:
        try:
            sftp_client.chdir(pre_path)  # Test if remote_path exists
        except IOError:
            sftp_client.mkdir(pre_path)  # Create remote_path

    pre_annotation_path = f"{pre_path}{pre_annotation_path}"
    pre_annotation_path_mrc = f"{pre_path}{pre_annotation_path_mrc}"
    current_pat_line_path = f"{pre_path}{current_pat_line_path}"
    current_pat_lines_path = current_pat_line_path

    if not remote_dump:
        Path(current_pat_annot_path).mkdir(parents=True, exist_ok=True)
        Path(pre_annotation_path_mrc).mkdir(parents=True, exist_ok=True)

    else:
        try:
            sftp_client.chdir(pre_annotation_path)  # Test if remote_path exists
        except IOError:
            sftp_client.mkdir(pre_annotation_path)  # Create remote_path

        try:
            sftp_client.chdir(pre_annotation_path_mrc)  # Test if remote_path exists
        except IOError:
            sftp_client.mkdir(pre_annotation_path_mrc)  # Create remote_path

        try:
            sftp_client.chdir(current_pat_line_path)  # Test if remote_path exists
        except IOError:
            sftp_client.mkdir(current_pat_line_path)  # Create remote_path
else:
    sftp_client = None

In [ ]:
def list_dir_wrapper(path, sftp_obj=None):
    # global sftp_client
    if remote_dump:
        if not share_sftp:
            ssh_client = paramiko.SSHClient()
            ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            ssh_client.connect(hostname=hostname, username=username, password=password)

            sftp_client = ssh_client.open_sftp()
            sftp_obj = sftp_client
        elif sftp_obj is None:
            sftp_obj = sftp_client

        res = sftp_obj.listdir(path)

        return res

    else:

        return os.listdir(path)

In [ ]:
def dump_results(file_data, path, sftp_obj=None):
    if remote_dump:
        if not share_sftp:
            ssh_client = paramiko.SSHClient()
            ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            ssh_client.connect(hostname=hostname, username=username, password=password)

            sftp_client = ssh_client.open_sftp()
            sftp_obj = sftp_client

        with sftp_obj.open(path, "w") as file:

            pickle.dump(file_data, file)
        if not share_sftp:
            sftp_obj.close()
            sftp_obj.close()

    else:
        with open(path, "wb") as f:
            pickle.dump(file_data, f)

In [ ]:
months = [x for x in range(1, 13)]
years = [x for x in range(1995, 2023)]
import itertools

combinations = list(itertools.product(years, months))
len(combinations)

In [ ]:
combinations[0:5]

In [ ]:
main_options = {
    "demo": True,
    "bmi": True,
    "bloods": True,
    "drugs": True,
    "diagnostics": True,
    "core_02": True,
    "bed": True,
    "vte_status": True,
    "hosp_site": True,
    "core_resus": True,
    "news": True,
    "annotations": True,
    "annotations_mrc": True,
}

In [ ]:
def get_demographics3(patlist, target_date_range):
    # print("get demo3 non batch call --debug")

    start_year, start_month, end_year, end_month = get_start_end_year_month(
        target_date_range
    )

    demo = cohort_searcher_with_terms_and_search(
        index_name="epr_documents",
        fields_list=[
            "client_idcode",
            "client_firstname",
            "client_lastname",
            "client_dob",
            "client_gendercode",
            "client_racecode",
            "client_deceaseddtm",
            "updatetime",
        ],
        term_name="client_idcode.keyword",
        entered_list=patlist,
        search_string=f"updatetime:[{start_year}-{start_month} TO {end_year}-{end_month}] ",
    )
    demo["updatetime"] = pd.to_datetime(demo["updatetime"], utc=True)
    demo = demo.sort_values(
        ["client_idcode", "updatetime"]
    )  # .drop_duplicates(subset = ["client_idcode"], keep = "last", inplace = True)
    if len(demo) > 1:
        try:

            return demo.iloc[-1].to_frame()
        except Exception as e:
            print(e)
    elif len(demo) == 1:
        return demo

    else:
        demo = pd.DataFrame(data=None, columns=None)
        demo["client_idcode"] = patlist
        return demo

In [ ]:
def get_start_end_year_month(target_date_range):

    start_year = target_date_range[0]
    start_month = target_date_range[1]
    end_year = start_year

    if target_date_range[1] == 12:
        end_year = start_year + 1
        end_month = 1
    else:
        end_year = target_date_range[0]
        end_month = start_month + 1

    if start_month < 10:
        start_month = "0" + str(start_month)

    if end_month < 10:
        end_month = "0" + str(end_month)

    return start_year, start_month, end_year, end_month

In [ ]:
# Should one impute from the global mean
# impute from local mean?
# correct stratification. by year and month...?
# by 6 months?
# dynamic and user spec time strat?
# encode month and year separately or every date/month
# c. 240
# with hfe annotation in vector, for positive can easily split by < >  date
# instead filter datetime at search level, since we want to produce individual vectors
# loop over vector production procedure for entry in date list for total 336 * npat = 336 * 20000 = 6,720,000 rows
# for c. 7000 included variables, n cells = 7000 * 6,720,000 = 47,040,000,000
# do we have updatetime for all data classes?

In [ ]:
def get_empty_date_vector():
    months = [x for x in range(1, 13)]
    years = [x for x in range(1995, 2023)]
    combinations = list(itertools.product(years, months))
    combinations = [str(item) + "_" + "date_time_stamp" for item in combinations]

    return pd.DataFrame(data=0.0, index=np.arange(1), columns=combinations).astype(
        float
    )  # untested float cast

In [ ]:
def convert_timestamp_to_tuple(timestamp):
    # parse the timestamp string into a datetime object
    dt = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%f%z")

    # extract the year and month from the datetime object
    year = dt.year
    month = dt.month

    # return the tuple of year and month
    return (year, month)

In [ ]:
def enum_target_date_vector(target_date_range, current_pat_client_id_code):

    empty_date_vector = get_empty_date_vector()

    empty_date_vector.at[0, str(target_date_range) + "_date_time_stamp"] = 1

    empty_date_vector["client_idcode"] = current_pat_client_id_code

    return empty_date_vector

In [ ]:
# Problems...:

# post code field map code to region for feature
# Religion code
# Language code
# occuptation code not specified
# country of birth empty
# marital status, populated but relevance?
# title code, populated
# address city populated
#
# OR Weight (kg)
# OR \"Height in cm\"

In [ ]:
# Good candidates:
# City of birth many results
#

In [ ]:
print(f"remote_dump {remote_dump}")
print(pre_annotation_path)
print(pre_annotation_path_mrc)
print(current_pat_line_path)

In [ ]:
def get_current_pat_annotations_mrc_cs(
    current_pat_client_id_code, target_date_range, pat_batch, sftp_obj=None
):
    global start_time

    #     current_annot_file_path = pre_annotation_path_mrc + current_pat_client_id_code+"_"+str(target_date_range)

    current_annot_file_path = (
        pre_annotation_path_mrc
        + current_pat_client_id_code
        + "/"
        + current_pat_client_id_code
        + "_"
        + str(target_date_range)
    )

    file_exists = exist_check(current_annot_file_path, sftp_obj)

    if not file_exists:

        start_year, start_month, end_year, end_month = get_start_end_year_month(
            target_date_range
        )

        if batch_mode:
            current_pat_docs = filter_dataframe_by_timestamp(
                pat_batch,
                start_year,
                start_month,
                end_year,
                end_month,
                "observationdocument_recordeddtm",
            )

        else:

            current_pat_docs = cohort_searcher_with_terms_and_search(
                index_name="observations",
                fields_list="""observation_guid client_idcode	obscatalogmasteritem_displayname	observation_valuetext_analysed	observationdocument_recordeddtm clientvisit_visitidcode""".split(),
                term_name="client_idcode.keyword",
                entered_list=[current_pat_client_id_code],
                search_string='obscatalogmasteritem_displayname:("AoMRC_ClinicalSummary_FT") AND '
                + f"observationdocument_recordeddtm:[{start_year}-{start_month} TO {end_year}-{end_month}]",
            )

        n_docs_to_annotate = len(current_pat_docs)
        update_pbar(
            current_pat_client_id_code + "_" + str(target_date_range),
            start_time,
            5,
            "annotations_mrc",
            n_docs_to_annotate=n_docs_to_annotate,
        )
    else:
        n_docs_to_annotate = "Reading preannotated mrc..."
    annotation_map = {
        "True": 1,
        "Presence": 1,
        "Recent": 1,
        "Past": 0,
        "Subject/Experiencer": 1,
        "Other": 0,
        "Hypothetical": 0,
        "Patient": 1,
    }

    # remove filter from cdb?
    # print("getting annotations")

    #     file_exists = exists(pre_annotation_path_mrc + current_pat_client_id_code)

    if not file_exists:
        with io.capture_output():
            pats_anno_annotations = cat.get_entities_multi_texts(
                current_pat_docs["observation_valuetext_analysed"].dropna()
            )
            # , n_process=1

            dump_results(pats_anno_annotations, current_annot_file_path, sftp_obj)

    #                 with open(pre_annotation_path_mrc + current_pat_client_id_code, 'wb') as f:
    #                     pickle.dump(pats_anno_annotations, f)

    else:
        if remote_dump:
            if not share_sftp:
                ssh_client = paramiko.SSHClient()
                ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
                ssh_client.connect(
                    hostname=hostname, username=username, password=password
                )

                sftp_client = ssh_client.open_sftp()
                sftp_obj = sftp_client

            with sftp_obj.open(current_annot_file_path, "r") as file:

                pats_anno_annotations = pickle.load(file)

            if not share_sftp:
                sftp_obj.close()
                sftp_obj.close()

        else:
            with open(current_annot_file_path, "rb") as f:
                pats_anno_annotations = pickle.load(f)

    n_docs_to_annotate = len(pats_anno_annotations)

    # print(f"Annotated {current_pat_client_id_code}")
    # length of chars in documents summed
    # average number of documents as a divisor for mention counts
    # we want to keep the fact that lots of documents is a bad sign...
    # Lots of mentions of something could indicate severity etc

    # pats_anno_annotations = cat.get_entities(current_pat_docs['body_analysed'])
    update_pbar(
        current_pat_client_id_code + "_" + str(target_date_range),
        start_time,
        5,
        "annotations_mrc",
        n_docs_to_annotate=n_docs_to_annotate,
    )

    sum_count = 0
    for i in range(0, len(pats_anno_annotations)):
        sum_count = sum_count + len(list(pats_anno_annotations[i]["entities"].keys()))

    sum_count_index_list = [x for x in range(0, sum_count)]
    all_doc_entities = {"entities": dict.fromkeys(sum_count_index_list, {})}
    sum_count_index = 0
    for i in range(0, len(pats_anno_annotations)):

        key_list = list(pats_anno_annotations[i]["entities"].keys())
        for j in range(0, len(key_list)):

            all_doc_entities["entities"][sum_count_index] = pats_anno_annotations[i][
                "entities"
            ].get(key_list[j])
            sum_count_index = sum_count_index + 1

    pats_anno_annotations = all_doc_entities

    all_cui_list = []

    all_meta_anno = False
    confidence_threshold_presence = 0.8
    confidence_threshold_concept_accuracy = 0.8

    cui_list_pretty_names = []
    doc_keys = list(pats_anno_annotations.keys())
    for i in range(0, len(doc_keys)):
        current_pats_entry = pats_anno_annotations.get("entities")
        current_pats_entry_keys = list(pats_anno_annotations.get("entities").keys())
        for j in range(0, len(current_pats_entry_keys)):
            all_cui_list.append(
                current_pats_entry.get(current_pats_entry_keys[j])["cui"]
            )
            cui_list_pretty_names.append(
                current_pats_entry.get(current_pats_entry_keys[j])["pretty_name"]
            )
    # print("len(all_cui_list)", len(all_cui_list))
    # print("len(set(all_cui_list))", len(set(all_cui_list)))

    cui_list = all_cui_list

    #     cui_list_pretty_names = []
    #     for i in range(0, len(cui_list)):
    #         cui_list_pretty_names.append(cat.cdb.cui2preferred_name.get(cui_list[i]))

    # print("len(set(cui_list_pretty_names))", len(set(cui_list_pretty_names)))

    cui_list_pretty_names = list(set(cui_list_pretty_names))

    # cui_list_pretty_names.remove(None)

    cui_list_pretty_names_meta_list = []
    if all_meta_anno:
        [x + "_meta" for x in cui_list_pretty_names]

        cui_list_pretty_names_meta_list = []

        meta_key_list = ["Time", "Presence", "Subject/Experiencer"]

        meta_sub_key_list = ["value"]  # ,'confidence', 'name']

        for i in range(0, len(cui_list_pretty_names)):
            for j in range(0, len(meta_key_list)):
                for k in range(0, len(meta_sub_key_list)):
                    cui_list_pretty_names_meta_list.append(
                        cui_list_pretty_names[i]
                        + "_"
                        + meta_key_list[j]
                        + "_"
                        + meta_sub_key_list[k]
                    )

        print(
            "len(set(cui_list_pretty_names_meta_list))",
            len(set(cui_list_pretty_names_meta_list)),
        )

    cui_list_pretty_names_count_list = []
    for i in range(0, len(cui_list_pretty_names)):
        cui_list_pretty_names_count_list.append(
            cui_list_pretty_names[i] + "_count_mrc_cs"
        )
        cui_list_pretty_names_count_list.append(
            cui_list_pretty_names[i] + "_count_subject_present_mrc_cs"
        )

        if negated_presence_annotations:
            cui_list_pretty_names_count_list.append(
                cui_list_pretty_names[i] + "_count_subject_not_present_mrc_cs"
            )
            cui_list_pretty_names_count_list.append(
                cui_list_pretty_names[i] + "_count_relative_not_present_mrc_cs"
            )

    all_columns_to_append = cui_list_pretty_names_count_list
    if all_meta_anno:
        all_columns_to_append.append(cui_list_pretty_names_meta_list)

    dummy_data = np.empty((1, len(all_columns_to_append)))
    dummy_data[:] = np.nan
    df_pat_entry = pd.DataFrame(data=dummy_data, columns=all_columns_to_append)

    # df = pd.read_csv(file_name) #call outside
    df_pat = df_pat_entry.copy()
    df_pat["client_idcode"] = current_pat_client_id_code
    df_pat["n_docs"] = n_docs_to_annotate

    df_pat.reset_index(inplace=True)
    # df_pat['n'] = [i for i in range(0, len(df_pat))]
    df_pat = df_pat[["client_idcode"]].copy()
    df_pat_target = df_pat.copy(deep=True)
    # print("Reindexing df_pat_target")
    df_pat_target = df_pat_target.reindex(
        list(df_pat_target.columns) + all_columns_to_append, axis=1
    )

    a = list(df_pat_target.columns)
    b = cui_list_pretty_names_meta_list

    # print("filling df pat target with nans")
    df_pat_target[[x for x in a if (x not in b)]] = df_pat_target[
        [x for x in a if (x not in b)]
    ].fillna(0)

    # break
    df_pat_target["client_idcode"].iloc[0] = current_pat_client_id_code

    df_pat_target = df_pat_target.copy()

    [x for x in range(0, len(df_pat_target))]

    list(cui_list_pretty_names_meta_list)

    df_pat_target = df_pat_target.copy()  # [0:1]

    entry_counter = 0
    meta_counter = 0
    i = 0
    # print("Starting annotation frame builder...")
    # for i in tqdm(range(0, len(df_pat_target))):
    # ci = df_pat_target['client_idcode'].iloc[i]

    # annotations = cat.get_entities(current_pat_docs['body_analysed']) #docs.get(ci)
    annotations = pats_anno_annotations

    if annotations is not None:
        annotation_keys = list(annotations["entities"].keys())

        for j in range(0, len(annotation_keys)):

            cui = annotations["entities"][annotation_keys[j]].get("cui")
            if cui in cui_list:
                current_col_name = annotations["entities"][annotation_keys[j]].get(
                    "pretty_name"
                )
                current_col_meta = annotations["entities"][annotation_keys[j]].get(
                    "meta_anns"
                )

                df_pat_target.at[i, current_col_name + "_count_mrc_cs"] = (
                    df_pat_target.loc[i][current_col_name + "_count_mrc_cs"] + 1
                )

                if current_col_meta is not None:

                    if (
                        current_col_meta["Presence"]["value"] == "True"
                        and current_col_meta["Subject/Experiencer"]["value"]
                        == "Patient"
                        and current_col_meta["Presence"]["confidence"]
                        > confidence_threshold_presence
                        and current_col_meta["Subject/Experiencer"]["confidence"]
                        > confidence_threshold_presence
                        and annotations["entities"][annotation_keys[j]]["acc"]
                        > confidence_threshold_concept_accuracy
                    ):

                        df_pat_target.at[
                            i, current_col_name + "_count_subject_present_mrc_cs"
                        ] = (
                            df_pat_target.loc[i][
                                current_col_name + "_count_subject_present_mrc_cs"
                            ]
                            + 1
                        )

                    elif (
                        current_col_meta["Presence"]["value"] == "True"
                        and current_col_meta["Subject/Experiencer"]["value"]
                        == "Relative"
                        and current_col_meta["Presence"]["confidence"]
                        > confidence_threshold_presence
                        and current_col_meta["Subject/Experiencer"]["confidence"]
                        > confidence_threshold_presence
                        and annotations["entities"][annotation_keys[j]]["acc"]
                        > confidence_threshold_concept_accuracy
                    ):
                        if (
                            current_col_name + "_count_relative_present_mrc_cs"
                            in df_pat_target.columns
                        ):

                            df_pat_target.at[
                                i, current_col_name + "_count_relative_present_mrc_cs"
                            ] = (
                                df_pat_target.loc[i][
                                    current_col_name + "_count_relative_present_mrc_cs"
                                ]
                                + 1
                            )

                        else:
                            df_pat_target[
                                current_col_name + "_count_relative_present_mrc_cs"
                            ] = 1

                    if negated_presence_annotations:

                        if current_col_meta is not None:

                            if (
                                current_col_meta["Presence"]["value"] == "False"
                                and current_col_meta["Subject/Experiencer"]["value"]
                                == "Patient"
                                and current_col_meta["Presence"]["confidence"]
                                > confidence_threshold_presence
                                and current_col_meta["Subject/Experiencer"][
                                    "confidence"
                                ]
                                > confidence_threshold_presence
                                and annotations["entities"][annotation_keys[j]]["acc"]
                                > confidence_threshold_concept_accuracy
                            ):

                                df_pat_target.at[
                                    i,
                                    current_col_name
                                    + "_count_subject_not_present_mrc_cs",
                                ] = (
                                    df_pat_target.loc[i][
                                        current_col_name
                                        + "_count_subject_not_present_mrc_cs"
                                    ]
                                    + 1
                                )

                            elif (
                                current_col_meta["Presence"]["value"] == "True"
                                and current_col_meta["Subject/Experiencer"]["value"]
                                == "Relative"
                                and current_col_meta["Presence"]["confidence"]
                                > confidence_threshold_presence
                                and current_col_meta["Subject/Experiencer"][
                                    "confidence"
                                ]
                                > confidence_threshold_presence
                                and annotations["entities"][annotation_keys[j]]["acc"]
                                > confidence_threshold_concept_accuracy
                            ):
                                if (
                                    current_col_name
                                    + "_count_relative_not_present_mrc_cs"
                                    in df_pat_target.columns
                                ):

                                    df_pat_target.at[
                                        i,
                                        current_col_name
                                        + "_count_relative_not_present_mrc_cs",
                                    ] = (
                                        df_pat_target.loc[i][
                                            current_col_name
                                            + "_count_relative_not_present_mrc_cs"
                                        ]
                                        + 1
                                    )

                                else:
                                    df_pat_target[
                                        current_col_name
                                        + "_count_relative_not_present_mrc_cs"
                                    ] = 1

                    else:
                        # OLD: set to nan instead of zero for impute and medcat precision persistence
                        # Dont set anything here as this will overwrite existing 1 entries?
                        pass
                        # df_pat_target.at[i, current_col_name+"_count_subject_present"] = np.nan

                if all_meta_anno:
                    if len(list(current_col_meta.keys())) > 0:

                        for key in current_col_meta.keys():

                            sub_anot = current_col_meta.get(key)
                            if len(list(sub_anot.keys())) > 0:
                                for sub_key in sub_anot.keys():
                                    if sub_key in meta_sub_key_list:
                                        try:

                                            key_result = sub_anot.get(sub_key)
                                            if type(key_result) is str:
                                                key_result = annotation_map.get(
                                                    key_result
                                                )

                                            # print(current_col_name+'_'+str(key)+'_'+str(sub_key), key_result, sub_anot.get(sub_key))

                                            df_pat_target.at[
                                                i,
                                                current_col_name
                                                + "_"
                                                + str(key)
                                                + "_"
                                                + str(sub_key),
                                            ] = key_result

                                            meta_counter = meta_counter + 1
                                        except Exception as e:
                                            print(e)
                                            pass

                meta_counter = meta_counter + 1
                entry_counter = entry_counter + 1

            else:
                pass
    update_pbar(
        current_pat_client_id_code + "_" + str(target_date_range),
        start_time,
        5,
        "annotations_mrc",
        n_docs_to_annotate=n_docs_to_annotate,
    )
    # df_pat_target.drop("n", axis=1, inplace=True)

    # df_pat_target.to_csv(entry_file_name)
    # print(f"Made {entry_counter} entry_counter  entries")
    # print(f"Made {meta_counter} meta_counter entries")
    # print("done")
    return df_pat_target

In [ ]:
def get_current_pat_annotations(
    current_pat_client_id_code, target_date_range, pat_batch, sftp_obj=None
):
    global start_time

    #     current_annotation_file_path = pre_annotation_path + current_pat_client_id_code+"_"+str(target_date_range)

    current_annotation_file_path = (
        pre_annotation_path
        + current_pat_client_id_code
        + "/"
        + current_pat_client_id_code
        + "_"
        + str(target_date_range)
    )

    file_exists = exist_check(current_annotation_file_path, sftp_obj)

    if not file_exists:

        start_year, start_month, end_year, end_month = get_start_end_year_month(
            target_date_range
        )

        if batch_mode:
            current_pat_docs = filter_dataframe_by_timestamp(
                pat_batch, start_year, start_month, end_year, end_month, "updatetime"
            )

        else:

            current_pat_docs = cohort_searcher_with_terms_and_search(
                index_name="epr_documents",
                fields_list="""client_idcode document_guid	document_description	body_analysed updatetime clientvisit_visitidcode""".split(),
                term_name="client_idcode.keyword",
                entered_list=[current_pat_client_id_code],
                search_string=f"updatetime:[{start_year}-{start_month} TO {end_year}-{end_month}] ",
            )

        n_docs_to_annotate = len(current_pat_docs)
        update_pbar(
            current_pat_client_id_code + "_" + str(target_date_range),
            start_time,
            5,
            "annotations",
            n_docs_to_annotate=n_docs_to_annotate,
        )

    else:
        n_docs_to_annotate = "Reading preannotated..."

    annotation_map = {
        "True": 1,
        "Presence": 1,
        "Recent": 1,
        "Past": 0,
        "Subject/Experiencer": 1,
        "Other": 0,
        "Hypothetical": 0,
        "Patient": 1,
    }

    # remove filter from cdb?
    # print("getting annotations")

    #     file_exists = exists(pre_annotation_path + current_pat_client_id_code)

    if not file_exists:
        with io.capture_output():
            pats_anno_annotations = cat.get_entities_multi_texts(
                current_pat_docs["body_analysed"].dropna()
            )
            # , n_process=1
        if store_annot:
            dump_results(pats_anno_annotations, current_annotation_file_path, sftp_obj)

    else:
        if remote_dump:
            if not share_sftp:
                ssh_client = paramiko.SSHClient()
                ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
                ssh_client.connect(
                    hostname=hostname, username=username, password=password
                )

                sftp_client = ssh_client.open_sftp()
                sftp_obj = sftp_client

            with sftp_obj.open(current_annotation_file_path, "r") as file:

                pats_anno_annotations = pickle.load(file)

            if not share_sftp:
                sftp_obj.close()
                sftp_obj.close()

        else:

            with open(current_annotation_file_path, "rb") as f:
                pats_anno_annotations = pickle.load(f)

    n_docs_to_annotate = len(pats_anno_annotations)

    # print(f"Annotated {current_pat_client_id_code}")
    # length of chars in documents summed
    # average number of documents as a divisor for mention counts
    # we want to keep the fact that lots of documents is a bad sign...
    # Lots of mentions of something could indicate severity etc

    # pats_anno_annotations = cat.get_entities(current_pat_docs['body_analysed'])
    update_pbar(
        current_pat_client_id_code + "_" + str(target_date_range),
        start_time,
        5,
        "annotations",
        n_docs_to_annotate=n_docs_to_annotate,
    )

    sum_count = 0
    for i in range(0, len(pats_anno_annotations)):
        sum_count = sum_count + len(list(pats_anno_annotations[i]["entities"].keys()))

    sum_count_index_list = [x for x in range(0, sum_count)]
    all_doc_entities = {"entities": dict.fromkeys(sum_count_index_list, {})}
    sum_count_index = 0
    for i in range(0, len(pats_anno_annotations)):

        key_list = list(pats_anno_annotations[i]["entities"].keys())
        for j in range(0, len(key_list)):

            all_doc_entities["entities"][sum_count_index] = pats_anno_annotations[i][
                "entities"
            ].get(key_list[j])
            sum_count_index = sum_count_index + 1

    pats_anno_annotations = all_doc_entities

    all_cui_list = []

    all_meta_anno = False
    confidence_threshold_presence = 0.8
    confidence_threshold_concept_accuracy = 0.8

    cui_list_pretty_names = []
    doc_keys = list(pats_anno_annotations.keys())
    for i in range(0, len(doc_keys)):
        current_pats_entry = pats_anno_annotations.get("entities")
        current_pats_entry_keys = list(pats_anno_annotations.get("entities").keys())
        for j in range(0, len(current_pats_entry_keys)):
            all_cui_list.append(
                current_pats_entry.get(current_pats_entry_keys[j])["cui"]
            )
            cui_list_pretty_names.append(
                current_pats_entry.get(current_pats_entry_keys[j])["pretty_name"]
            )
    # print("len(all_cui_list)", len(all_cui_list))
    # print("len(set(all_cui_list))", len(set(all_cui_list)))

    cui_list = all_cui_list

    #     cui_list_pretty_names = []
    #     for i in range(0, len(cui_list)):
    #         cui_list_pretty_names.append(cat.cdb.cui2preferred_name.get(cui_list[i]))

    # print("len(set(cui_list_pretty_names))", len(set(cui_list_pretty_names)))

    cui_list_pretty_names = list(set(cui_list_pretty_names))

    # cui_list_pretty_names.remove(None)

    cui_list_pretty_names_meta_list = []
    if all_meta_anno:
        [x + "_meta" for x in cui_list_pretty_names]

        cui_list_pretty_names_meta_list = []

        meta_key_list = ["Time", "Presence", "Subject/Experiencer"]

        meta_sub_key_list = ["value"]  # ,'confidence', 'name']

        for i in range(0, len(cui_list_pretty_names)):
            for j in range(0, len(meta_key_list)):
                for k in range(0, len(meta_sub_key_list)):
                    cui_list_pretty_names_meta_list.append(
                        cui_list_pretty_names[i]
                        + "_"
                        + meta_key_list[j]
                        + "_"
                        + meta_sub_key_list[k]
                    )

        print(
            "len(set(cui_list_pretty_names_meta_list))",
            len(set(cui_list_pretty_names_meta_list)),
        )

    cui_list_pretty_names_count_list = []
    for i in range(0, len(cui_list_pretty_names)):
        cui_list_pretty_names_count_list.append(cui_list_pretty_names[i] + "_count")
        cui_list_pretty_names_count_list.append(
            cui_list_pretty_names[i] + "_count_subject_present"
        )

        if negated_presence_annotations:
            cui_list_pretty_names_count_list.append(
                cui_list_pretty_names[i] + "_count_subject_not_present"
            )
            cui_list_pretty_names_count_list.append(
                cui_list_pretty_names[i] + "_count_relative_not_present"
            )

    all_columns_to_append = cui_list_pretty_names_count_list
    if all_meta_anno:
        all_columns_to_append.append(cui_list_pretty_names_meta_list)

    dummy_data = np.empty((1, len(all_columns_to_append)))
    dummy_data[:] = np.nan
    df_pat_entry = pd.DataFrame(data=dummy_data, columns=all_columns_to_append)

    # df = pd.read_csv(file_name) #call outside
    df_pat = df_pat_entry.copy()
    df_pat["client_idcode"] = current_pat_client_id_code
    df_pat["n_docs"] = n_docs_to_annotate

    df_pat.reset_index(inplace=True)
    # df_pat['n'] = [i for i in range(0, len(df_pat))]
    df_pat = df_pat[["client_idcode"]].copy()
    df_pat_target = df_pat.copy(deep=True)
    # print("Reindexing df_pat_target")
    df_pat_target = df_pat_target.reindex(
        list(df_pat_target.columns) + all_columns_to_append, axis=1
    )

    a = list(df_pat_target.columns)
    b = cui_list_pretty_names_meta_list

    # print("filling df pat target with nans")
    df_pat_target[[x for x in a if (x not in b)]] = df_pat_target[
        [x for x in a if (x not in b)]
    ].fillna(0)

    # break
    df_pat_target["client_idcode"].iloc[0] = current_pat_client_id_code

    df_pat_target = df_pat_target.copy()

    [x for x in range(0, len(df_pat_target))]

    list(cui_list_pretty_names_meta_list)

    df_pat_target = df_pat_target.copy()  # [0:1]

    entry_counter = 0
    meta_counter = 0
    i = 0
    # print("Starting annotation frame builder...")
    # for i in tqdm(range(0, len(df_pat_target))):
    # ci = df_pat_target['client_idcode'].iloc[i]

    # annotations = cat.get_entities(current_pat_docs['body_analysed']) #docs.get(ci)
    annotations = pats_anno_annotations

    if annotations is not None:
        annotation_keys = list(annotations["entities"].keys())

        for j in range(0, len(annotation_keys)):

            cui = annotations["entities"][annotation_keys[j]].get("cui")
            if cui in cui_list:
                current_col_name = annotations["entities"][annotation_keys[j]].get(
                    "pretty_name"
                )
                current_col_meta = annotations["entities"][annotation_keys[j]].get(
                    "meta_anns"
                )

                df_pat_target.at[i, current_col_name + "_count"] = (
                    df_pat_target.loc[i][current_col_name + "_count"] + 1
                )

                if current_col_meta is not None:

                    if (
                        current_col_meta["Presence"]["value"] == "True"
                        and current_col_meta["Subject/Experiencer"]["value"]
                        == "Patient"
                        and current_col_meta["Presence"]["confidence"]
                        > confidence_threshold_presence
                        and current_col_meta["Subject/Experiencer"]["confidence"]
                        > confidence_threshold_presence
                        and annotations["entities"][annotation_keys[j]]["acc"]
                        > confidence_threshold_concept_accuracy
                    ):

                        df_pat_target.at[
                            i, current_col_name + "_count_subject_present"
                        ] = (
                            df_pat_target.loc[i][
                                current_col_name + "_count_subject_present"
                            ]
                            + 1
                        )

                    elif (
                        current_col_meta["Presence"]["value"] == "True"
                        and current_col_meta["Subject/Experiencer"]["value"]
                        == "Relative"
                        and current_col_meta["Presence"]["confidence"]
                        > confidence_threshold_presence
                        and current_col_meta["Subject/Experiencer"]["confidence"]
                        > confidence_threshold_presence
                        and annotations["entities"][annotation_keys[j]]["acc"]
                        > confidence_threshold_concept_accuracy
                    ):
                        if (
                            current_col_name + "_count_relative_present"
                            in df_pat_target.columns
                        ):

                            df_pat_target.at[
                                i, current_col_name + "_count_relative_present"
                            ] = (
                                df_pat_target.loc[i][
                                    current_col_name + "_count_relative_present"
                                ]
                                + 1
                            )

                        else:
                            df_pat_target[
                                current_col_name + "_count_relative_present"
                            ] = 1

                    if negated_presence_annotations:

                        if current_col_meta is not None:

                            if (
                                current_col_meta["Presence"]["value"] == "False"
                                and current_col_meta["Subject/Experiencer"]["value"]
                                == "Patient"
                                and current_col_meta["Presence"]["confidence"]
                                > confidence_threshold_presence
                                and current_col_meta["Subject/Experiencer"][
                                    "confidence"
                                ]
                                > confidence_threshold_presence
                                and annotations["entities"][annotation_keys[j]]["acc"]
                                > confidence_threshold_concept_accuracy
                            ):

                                df_pat_target.at[
                                    i, current_col_name + "_count_subject_not_present"
                                ] = (
                                    df_pat_target.loc[i][
                                        current_col_name + "_count_subject_not_present"
                                    ]
                                    + 1
                                )

                            elif (
                                current_col_meta["Presence"]["value"] == "True"
                                and current_col_meta["Subject/Experiencer"]["value"]
                                == "Relative"
                                and current_col_meta["Presence"]["confidence"]
                                > confidence_threshold_presence
                                and current_col_meta["Subject/Experiencer"][
                                    "confidence"
                                ]
                                > confidence_threshold_presence
                                and annotations["entities"][annotation_keys[j]]["acc"]
                                > confidence_threshold_concept_accuracy
                            ):
                                if (
                                    current_col_name + "_count_relative_not_present"
                                    in df_pat_target.columns
                                ):

                                    df_pat_target.at[
                                        i,
                                        current_col_name
                                        + "_count_relative_not_present",
                                    ] = (
                                        df_pat_target.loc[i][
                                            current_col_name
                                            + "_count_relative_not_present"
                                        ]
                                        + 1
                                    )

                                else:
                                    df_pat_target[
                                        current_col_name + "_count_relative_not_present"
                                    ] = 1

                    else:
                        # OLD: set to nan instead of zero for impute and medcat precision persistence
                        # Dont set anything here as this will overwrite existing 1 entries?
                        pass
                        # df_pat_target.at[i, current_col_name+"_count_subject_present"] = np.nan

                if all_meta_anno:
                    if len(list(current_col_meta.keys())) > 0:

                        for key in current_col_meta.keys():

                            sub_anot = current_col_meta.get(key)
                            if len(list(sub_anot.keys())) > 0:
                                for sub_key in sub_anot.keys():
                                    if sub_key in meta_sub_key_list:
                                        try:

                                            key_result = sub_anot.get(sub_key)
                                            if type(key_result) is str:
                                                key_result = annotation_map.get(
                                                    key_result
                                                )

                                            # print(current_col_name+'_'+str(key)+'_'+str(sub_key), key_result, sub_anot.get(sub_key))

                                            df_pat_target.at[
                                                i,
                                                current_col_name
                                                + "_"
                                                + str(key)
                                                + "_"
                                                + str(sub_key),
                                            ] = key_result

                                            meta_counter = meta_counter + 1
                                        except Exception as e:
                                            print(e)
                                            pass

                meta_counter = meta_counter + 1
                entry_counter = entry_counter + 1

            else:
                pass
    update_pbar(
        current_pat_client_id_code + "_" + str(target_date_range),
        start_time,
        5,
        "annotations",
        n_docs_to_annotate=n_docs_to_annotate,
    )
    # df_pat_target.drop("n", axis=1, inplace=True)

    # df_pat_target.to_csv(entry_file_name)
    # print(f"Made {entry_counter} entry_counter  entries")
    # print(f"Made {meta_counter} meta_counter entries")
    # print("done")
    return df_pat_target

In [ ]:
def update_pbar(
    current_pat_client_id_code, start_time, stage_int, stage_str, **n_docs_to_annotate
):
    global colour_val
    global t
    global skipped_counter

    # global skipped_counter
    # colour_val = color_bars[stage_int] + stage_str
    colour_val = Fore.GREEN + Style.BRIGHT + stage_str

    if multi_process:

        counter_disp = skipped_counter.value

    else:
        counter_disp = skipped_counter

    t.set_description(
        f"s: {counter_disp} | {current_pat_client_id_code} | task: {colour_val} | {n_docs_to_annotate}"
    )
    if (time.time() - start_time) > slow_execution_threshold_low:
        t.colour = Fore.YELLOW
        colour_val = Fore.YELLOW + stage_str
        t.set_description(
            f"s: {counter_disp} | {current_pat_client_id_code} | task: {colour_val} | {n_docs_to_annotate}"
        )

    elif (time.time() - start_time) > slow_execution_threshold_high:
        t.colour = Fore.RED + Style.BRIGHT
        colour_val = Fore.RED + Style.BRIGHT + stage_str
        t.set_description(
            f"s: {counter_disp} | {current_pat_client_id_code} | task: {colour_val} | {n_docs_to_annotate}"
        )

    elif (time.time() - start_time) > slow_execution_threshold_extreme:
        t.colour = Fore.RED + Style.DIM
        colour_val = Fore.RED + Style.DIM + stage_str
        t.set_description(
            f"s: {counter_disp} | {current_pat_client_id_code} | task: {colour_val} | {n_docs_to_annotate}"
        )

    else:
        t.colour = Fore.GREEN + Style.DIM
        colour_val = Fore.GREEN + Style.DIM + stage_str
        t.set_description(
            f"s: {counter_disp} | {current_pat_client_id_code} | task: {colour_val} | {n_docs_to_annotate}"
        )

    t.refresh()

In [ ]:
current_pat_lines_path

In [ ]:
stripped_list_start = [
    x.replace(".csv", "") for x in list_dir_wrapper(current_pat_lines_path, sftp_client)
]

In [ ]:
len(stripped_list_start)

In [ ]:
stripped_list = [
    x.replace(".csv", "") for x in list_dir_wrapper(current_pat_lines_path, sftp_client)
]

In [ ]:
def get_current_pat_annotations_to_file(current_pat_client_id_code, target_date_range):
    start_time = time.time()

    file_exists = exist_check(
        pre_annotation_path + current_pat_client_id_code + "_" + str(target_date_range)
    )

    if not file_exists:

        start_year, start_month, end_year, end_month = get_start_end_year_month(
            target_date_range
        )

        current_pat_docs = cohort_searcher_with_terms_and_search(
            index_name="epr_documents",
            fields_list="""client_idcode document_guid	document_description	body_analysed updatetime clientvisit_visitidcode""".split(),
            term_name="client_idcode.keyword",
            entered_list=[current_pat_client_id_code],
            search_string=f"updatetime:[{start_year}-{start_month} TO {end_year}-{end_month}] ",
        )

        n_docs_to_annotate = len(current_pat_docs)
        update_pbar(
            current_pat_client_id_code + "_" + str(target_date_range),
            start_time,
            5,
            "annotations",
            n_docs_to_annotate=n_docs_to_annotate,
        )


    # remove filter from cdb?
    # print("getting annotations")

    #     file_exists = exists(pre_annotation_path + current_pat_client_id_code)

    if not file_exists:
        with io.capture_output():
            pats_anno_annotations = cat.get_entities_multi_texts(
                current_pat_docs["body_analysed"].dropna()
            )
            # , n_process=1
        if store_annot:
            dump_results(
                pats_anno_annotations,
                pre_annotation_path
                + current_pat_client_id_code
                + "_"
                + str(target_date_range),
            )

In [ ]:
def filter_dataframe_by_timestamp(
    df, start_year, start_month, end_year, end_month, timestamp_string
):
    # Convert timestamp column to datetime format
    df[timestamp_string] = pd.to_datetime(df[timestamp_string], utc=True)

    # imputed from elastic. Mirror:
    start_day = 1
    end_day = 1
    hour = 23
    minute = 59
    second = 59

    # Filter based on year and month ranges
    filtered_df = df[
        (
            df[timestamp_string]
            >= str(
                datetime(start_year, int(start_month), start_day, hour, minute, second)
            )
        )
        & (
            df[timestamp_string]
            <= str(datetime(end_year, int(end_month), end_day, hour, minute, second))
        )
    ]

    return filtered_df

In [ ]:
def get_current_pat_annotations_mct_batch_to_file(
    current_pat_client_id_code,
    target_date_range,
    pat_doc_batch,
    sftp_obj=None,
    skip_check=False,
):
    start_time = time.time()

    current_annot_file_path = (
        pre_annotation_path_mrc
        + current_pat_client_id_code
        + "/"
        + current_pat_client_id_code
        + "_"
        + str(target_date_range)
    )

    if skip_check:
        file_exists = False
    else:
        file_exists = exist_check(current_annot_file_path, sftp_obj)

    if not file_exists:

        start_year, start_month, end_year, end_month = get_start_end_year_month(
            target_date_range
        )

        current_pat_docs = filter_dataframe_by_timestamp(
            pat_doc_batch,
            start_year,
            start_month,
            end_year,
            end_month,
            "observationdocument_recordeddtm",
        )

        n_docs_to_annotate = len(current_pat_docs)
        update_pbar(
            current_pat_client_id_code + "_" + str(target_date_range),
            start_time,
            5,
            "annotations_mct",
            n_docs_to_annotate=n_docs_to_annotate,
        )

    if not file_exists:
        with io.capture_output():
            pats_anno_annotations = cat.get_entities_multi_texts(
                current_pat_docs["observation_valuetext_analysed"].dropna()
            )
            # , n_process=1
        if store_annot:
            dump_results(pats_anno_annotations, current_annot_file_path, sftp_obj)

In [ ]:
def get_current_pat_annotations_batch_to_file(
    current_pat_client_id_code,
    target_date_range,
    pat_doc_batch,
    sftp_obj=None,
    skip_check=False,
):
    start_time = time.time()

    current_annotation_file_path = (
        pre_annotation_path
        + current_pat_client_id_code
        + "/"
        + current_pat_client_id_code
        + "_"
        + str(target_date_range)
    )

    if skip_check:
        file_exists = False
    else:
        file_exists = exist_check(current_annotation_file_path, sftp_obj)

    if not file_exists:

        start_year, start_month, end_year, end_month = get_start_end_year_month(
            target_date_range
        )

        current_pat_docs = filter_dataframe_by_timestamp(
            pat_doc_batch, start_year, start_month, end_year, end_month, "updatetime"
        )

        n_docs_to_annotate = len(current_pat_docs)
        update_pbar(
            current_pat_client_id_code + "_" + str(target_date_range),
            start_time,
            5,
            "annotations",
            n_docs_to_annotate=n_docs_to_annotate,
        )


    # remove filter from cdb?
    # print("getting annotations")

    #     file_exists = exists(pre_annotation_path + current_pat_client_id_code)

    if not file_exists:
        with io.capture_output():
            pats_anno_annotations = cat.get_entities_multi_texts(
                current_pat_docs["body_analysed"].dropna()
            )
            # , n_process=1
        if store_annot:
            dump_results(pats_anno_annotations, current_annotation_file_path, sftp_obj)

In [ ]:
def get_demographics3_batch(patlist, target_date_range, pat_batch):

    start_year, start_month, end_year, end_month = get_start_end_year_month(
        target_date_range
    )

    if batch_mode:

        demo = filter_dataframe_by_timestamp(
            pat_batch, start_year, start_month, end_year, end_month, "updatetime"
        )

    else:
        demo = cohort_searcher_with_terms_and_search(
            index_name="epr_documents",
            fields_list=[
                "client_idcode",
                "client_firstname",
                "client_lastname",
                "client_dob",
                "client_gendercode",
                "client_racecode",
                "client_deceaseddtm",
                "updatetime",
            ],
            term_name="client_idcode.keyword",
            entered_list=patlist,
            search_string=f"updatetime:[{start_year}-{start_month} TO {end_year}-{end_month}] ",
        )

    demo["updatetime"] = pd.to_datetime(demo["updatetime"], utc=True)
    demo = demo.sort_values(
        ["client_idcode", "updatetime"]
    )  # .drop_duplicates(subset = ["client_idcode"], keep = "last", inplace = True)

    # if more than one in the range return the nearest the end of the period
    if len(demo) > 1:
        try:
            # print("case1")
            return demo.tail(1)
            # return demo.iloc[-1].to_frame()
        except Exception as e:
            print(e)

    # if only one return it
    elif len(demo) == 1:
        return demo

    # otherwise return only the client id
    else:
        demo = pd.DataFrame(data=None, columns=None)
        demo["client_idcode"] = patlist
        return demo

In [ ]:
def get_demo(current_pat_client_id_code, target_date_range, pat_batch):

    current_pat_demo = get_demographics3_batch(
        [current_pat_client_id_code], target_date_range, pat_batch
    )

    # display(current_pat_demo)

    # print(len(current_pat_demo.columns))

    if len(current_pat_demo.columns) > 1:

        current_pat_demo = append_age_at_record_series(current_pat_demo)

        # demo_dataframe = pd.DataFrame(current_pat_demo).T

        # demo_dataframe.reset_index(inplace=True)

        demo_dataframe = current_pat_demo.copy()

        current_pat_demo = EthnicityAbstractor.abstractEthnicity(
            demo_dataframe,
            outputNameString="_census",
            ethnicityColumnString="client_racecode",
        )

        dummied_dummy_ethnicity_dataframe = pd.DataFrame(
            data=[np.zeros(5)],
            columns=[
                "census_white",
                "census_asian_or_asian_british",
                "census_black_african_caribbean_or_black_british",
                "census_mixed_or_multiple_ethnic_groups",
                "census_other_ethnic_group",
            ],
        )

        cen_res = pd.get_dummies(current_pat_demo["census"], prefix="census")

        dummied_dummy_ethnicity_dataframe[cen_res.columns[0]] = cen_res[
            cen_res.columns[0]
        ]

        abstrated_ethnicity_dummied = dummied_dummy_ethnicity_dataframe

        # abstrated_ethnicity_dummied = pd.concat([dummied_dummy_ethnicity_dataframe, pd.get_dummies(current_pat_demo['census'], prefix = 'census')], axis=1)

        current_pat_demo = pd.concat(
            [current_pat_demo.reset_index(), abstrated_ethnicity_dummied], axis=1
        )

        current_pat_demo.reset_index(inplace=True)

        # current_pat_demo

        sex_map = {"Male": 1, "Female": 0, "male": 1, "female": 0}

        #         def is_dead(line):
        #             try:
        #                 if(type(line)==str or type(line)==float):
        #                     return np.isnan(line)
        #                 else:
        #                     return line.isnull
        #             except Exception as e:
        #                 print(e)
        #                 print(type(line))
        #                 print(line)
        #         def is_dead(line):
        #             return line.isna()

        current_pat_demo["male"] = current_pat_demo["client_gendercode"].map(sex_map)

        # current_pat_demo['dead'] = current_pat_demo['client_deceaseddtm'].apply(is_dead)

        # current_pat_demo['dead'] = int(int((np.isnan(current_pat_demo['client_deceaseddtm'].iloc[0])))!=1)

        current_pat_demo["dead"] = int(
            type(current_pat_demo["client_deceaseddtm"]) == str
        )

        current_pat_demo = current_pat_demo[
            [
                "client_idcode",
                "male",
                "age",
                "dead",
                "census_white",
                "census_asian_or_asian_british",
                "census_black_african_caribbean_or_black_british",
                "census_mixed_or_multiple_ethnic_groups",
                "census_other_ethnic_group",
            ]
        ].copy()

        current_pat_demo["dead"] = current_pat_demo["dead"].astype(int)

        current_pat_demo["age"] = current_pat_demo["age"].astype(int)

        current_pat_demo["dead"] = current_pat_demo["dead"].astype(float)

        current_pat_demo["age"] = current_pat_demo["age"].astype(float)

        current_pat_demo["male"] = current_pat_demo["male"].astype(float)

        return current_pat_demo

In [ ]:
def get_smoking(current_pat_client_id_code, target_date_range, pat_batch):
    start_year, start_month, end_year, end_month = get_start_end_year_month(
        target_date_range
    )

    search_term = "CORE_SmokingStatus"

    if batch_mode:
        current_pat_raw = filter_dataframe_by_timestamp(
            pat_batch,
            start_year,
            start_month,
            end_year,
            end_month,
            "observationdocument_recordeddtm",
        )

    else:

        current_pat_raw = cohort_searcher_with_terms_and_search(
            index_name="observations",
            fields_list="""observation_guid client_idcode	obscatalogmasteritem_displayname	observation_valuetext_analysed	observationdocument_recordeddtm clientvisit_visitidcode""".split(),
            term_name="client_idcode.keyword",
            entered_list=[current_pat_client_id_code],
            #                                                                 search_string=f"obscatalogmasteritem_displayname:(\"{search_term}\")")
            search_string=f'obscatalogmasteritem_displayname:("{search_term}") AND '
            + f"observationdocument_recordeddtm:[{start_year}-{start_month} TO {end_year}-{end_month}]",
        )
    if len(current_pat_raw) == 0:

        features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        )

    features_data = current_pat_raw[
        current_pat_raw["obscatalogmasteritem_displayname"] == search_term
    ].copy()

    # screen and purge dud values
    # features_data =  features_data[(features_data['observation_valuetext_analysed'].astype(float)<20)& (features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    features_data.dropna(inplace=True)

    # -----------------------------------------------------------------

    features_data = current_pat_raw[
        current_pat_raw["obscatalogmasteritem_displayname"] == search_term
    ].copy()
    # features_data =  features_data[(features_data['observation_valuetext_analysed'].astype(float)<20)& (features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    # features_data.dropna(inplace=True)

    term = "smoking_status".lower()

    if len(features_data) > 0:
        features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        ).copy()

        value_array = features_data["observation_valuetext_analysed"].dropna()

        features[f"{term}_current"] = value_array.str.contains("Current Smoker").astype(
            int
        )
        features[f"{term}_non"] = value_array.str.contains("Non-Smoker").astype(int)

    else:
        features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        ).copy()
        features[f"{term}_current"] = np.nan
        features[f"{term}_non"] = np.nan

    return features

In [ ]:
def get_core_02(current_pat_client_id_code, target_date_range, pat_batch):

    start_year, start_month, end_year, end_month = get_start_end_year_month(
        target_date_range
    )

    search_term = "CORE_SpO2"

    if batch_mode:
        current_pat_raw = filter_dataframe_by_timestamp(
            pat_batch,
            start_year,
            start_month,
            end_year,
            end_month,
            "observationdocument_recordeddtm",
        )

    else:
        current_pat_raw = cohort_searcher_with_terms_and_search(
            index_name="observations",
            fields_list="""observation_guid client_idcode	obscatalogmasteritem_displayname	observation_valuetext_analysed	observationdocument_recordeddtm clientvisit_visitidcode""".split(),
            term_name="client_idcode.keyword",
            entered_list=[current_pat_client_id_code],
            search_string=f'obscatalogmasteritem_displayname:("{search_term}") AND '
            + f"observationdocument_recordeddtm:[{start_year}-{start_month} TO {end_year}-{end_month}]",
        )

    if len(current_pat_raw) == 0:

        features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        )

    features_data = current_pat_raw[
        current_pat_raw["obscatalogmasteritem_displayname"] == search_term
    ].copy()

    # screen and purge dud values
    # features_data =  features_data[(features_data['observation_valuetext_analysed'].astype(float)<20)& (features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    features_data.dropna(inplace=True)

    # -----------------------------------------------------------------

    features_data = current_pat_raw[
        current_pat_raw["obscatalogmasteritem_displayname"] == search_term
    ].copy()
    # features_data =  features_data[(features_data['observation_valuetext_analysed'].astype(float)<20)& (features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    # features_data.dropna(inplace=True)

    term = "core_sp_02".lower()

    if len(features_data) > 0:
        features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        ).copy()

        all_terms = list(
            features_data["observation_valuetext_analysed"].dropna().unique()
        )

        for term in all_terms:

            features[f'{term.replace("-", "_").replace("%", "pct")}'] = 1
            # features[f'{bed_term}'] = 1

    return features

In [ ]:
def get_bed(current_pat_client_id_code, target_date_range, pat_batch):

    start_year, start_month, end_year, end_month = get_start_end_year_month(
        target_date_range
    )

    search_term = "CORE_BedNumber3"

    if batch_mode:
        current_pat_raw = filter_dataframe_by_timestamp(
            pat_batch,
            start_year,
            start_month,
            end_year,
            end_month,
            "observationdocument_recordeddtm",
        )

    else:
        current_pat_raw = cohort_searcher_with_terms_and_search(
            index_name="observations",
            fields_list="""observation_guid client_idcode	obscatalogmasteritem_displayname	observation_valuetext_analysed	observationdocument_recordeddtm clientvisit_visitidcode""".split(),
            term_name="client_idcode.keyword",
            entered_list=[current_pat_client_id_code],
            search_string=f'obscatalogmasteritem_displayname:("{search_term}") AND '
            + f"observationdocument_recordeddtm:[{start_year}-{start_month} TO {end_year}-{end_month}]",
        )

    if len(current_pat_raw) == 0:

        features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        )

    features_data = current_pat_raw[
        current_pat_raw["obscatalogmasteritem_displayname"] == search_term
    ].copy()

    # screen and purge dud values
    # features_data =  features_data[(features_data['observation_valuetext_analysed'].astype(float)<20)& (features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    features_data.dropna(inplace=True)

    # -----------------------------------------------------------------

    features_data = current_pat_raw[
        current_pat_raw["obscatalogmasteritem_displayname"] == search_term
    ].copy()
    # features_data =  features_data[(features_data['observation_valuetext_analysed'].astype(float)<20)& (features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    # features_data.dropna(inplace=True)

    "bed".lower()

    if len(features_data) > 0:
        features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        ).copy()

        all_bed_terms = list(
            features_data["observation_valuetext_analysed"].dropna().unique()
        )

        for bed_term in all_bed_terms:

            features[f"bed_{bed_term}"] = 1

    return features

In [ ]:
def get_vte_status(current_pat_client_id_code, target_date_range, pat_batch):

    start_year, start_month, end_year, end_month = get_start_end_year_month(
        target_date_range
    )

    search_term = "CORE_VTE_STATUS"

    if batch_mode:
        current_pat_raw = filter_dataframe_by_timestamp(
            pat_batch,
            start_year,
            start_month,
            end_year,
            end_month,
            "observationdocument_recordeddtm",
        )

    else:
        current_pat_raw = cohort_searcher_with_terms_and_search(
            index_name="observations",
            fields_list="""observation_guid client_idcode	obscatalogmasteritem_displayname	observation_valuetext_analysed	observationdocument_recordeddtm clientvisit_visitidcode""".split(),
            term_name="client_idcode.keyword",
            entered_list=[current_pat_client_id_code],
            search_string=f'obscatalogmasteritem_displayname:("{search_term}") AND '
            + f"observationdocument_recordeddtm:[{start_year}-{start_month} TO {end_year}-{end_month}]",
        )

    if len(current_pat_raw) == 0:

        features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        )

    features_data = current_pat_raw[
        current_pat_raw["obscatalogmasteritem_displayname"] == search_term
    ].copy()

    # screen and purge dud values
    # features_data =  features_data[(features_data['observation_valuetext_analysed'].astype(float)<20)& (features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    features_data.dropna(inplace=True)

    # -----------------------------------------------------------------

    features_data = current_pat_raw[
        current_pat_raw["obscatalogmasteritem_displayname"] == search_term
    ].copy()
    # features_data =  features_data[(features_data['observation_valuetext_analysed'].astype(float)<20)& (features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    # features_data.dropna(inplace=True)

    term = "VTE_Status".lower()

    if len(features_data) > 0:
        features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        ).copy()

        di = {
            "High risk of VTE High risk of bleeding": 1,
            "High risk of VTE Low risk of bleeding": 0,
        }

        value_array = features_data["observation_valuetext_analysed"].map(di)

        value_array = value_array.astype(float)

        features[f"{term}_mean"] = value_array.mean()
        features[f"{term}_median"] = value_array.median()
        features[f"{term}_std"] = value_array.std()
        features[f"{term}_max"] = max(value_array)
        features[f"{term}_min"] = min(value_array)
        features[f"{term}_n"] = value_array.shape[0]
    else:
        features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        ).copy()
        features[f"{term}_mean"] = np.nan
        features[f"{term}_median"] = np.nan
        features[f"{term}_std"] = np.nan
        features[f"{term}_max"] = np.nan
        features[f"{term}_min"] = np.nan
        features[f"{term}_n"] = np.nan

    return features

In [ ]:
def get_hosp_site(current_pat_client_id_code, target_date_range, pat_batch):

    start_year, start_month, end_year, end_month = get_start_end_year_month(
        target_date_range
    )

    search_term = "CORE_HospitalSite"
    if batch_mode:
        current_pat_raw = filter_dataframe_by_timestamp(
            pat_batch,
            start_year,
            start_month,
            end_year,
            end_month,
            "observationdocument_recordeddtm",
        )

    else:
        current_pat_raw = cohort_searcher_with_terms_and_search(
            index_name="observations",
            fields_list="""observation_guid client_idcode	obscatalogmasteritem_displayname	observation_valuetext_analysed	observationdocument_recordeddtm clientvisit_visitidcode""".split(),
            term_name="client_idcode.keyword",
            entered_list=[current_pat_client_id_code],
            search_string=f'obscatalogmasteritem_displayname:("{search_term}") AND '
            + f"observationdocument_recordeddtm:[{start_year}-{start_month} TO {end_year}-{end_month}]",
        )

    if len(current_pat_raw) == 0:

        features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        )

    features_data = current_pat_raw[
        current_pat_raw["obscatalogmasteritem_displayname"] == search_term
    ].copy()

    # screen and purge dud values
    # features_data =  features_data[(features_data['observation_valuetext_analysed'].astype(float)<20)& (features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    features_data.dropna(inplace=True)

    # -----------------------------------------------------------------

    features_data = current_pat_raw[
        current_pat_raw["obscatalogmasteritem_displayname"] == search_term
    ].copy()
    # features_data =  features_data[(features_data['observation_valuetext_analysed'].astype(float)<20)& (features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    # features_data.dropna(inplace=True)

    term = "hosp_site".lower()

    if len(features_data) > 0:
        features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        ).copy()
        # value_array = features_data['observation_valuetext_analysed'].astype(float)
        value_array = features_data["observation_valuetext_analysed"].dropna()

        features[f"{term}_dh"] = value_array.str.contains("DH").astype(int)
        features[f"{term}_ph"] = value_array.str.contains("PRUH").astype(int)

    else:
        features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        ).copy()
        features[f"{term}_dh"] = np.nan
        features[f"{term}_ph"] = np.nan

    return features

In [ ]:
def get_core_resus(current_pat_client_id_code, target_date_range, pat_batch):

    start_year, start_month, end_year, end_month = get_start_end_year_month(
        target_date_range
    )

    if batch_mode:
        current_pat_raw = filter_dataframe_by_timestamp(
            pat_batch,
            start_year,
            start_month,
            end_year,
            end_month,
            "observationdocument_recordeddtm",
        )

    else:
        current_pat_raw = cohort_searcher_with_terms_and_search(
            index_name="observations",
            fields_list="""observation_guid client_idcode	obscatalogmasteritem_displayname	observation_valuetext_analysed	observationdocument_recordeddtm clientvisit_visitidcode""".split(),
            term_name="client_idcode.keyword",
            entered_list=[current_pat_client_id_code],
            search_string='obscatalogmasteritem_displayname:("CORE_RESUS_STATUS") AND '
            + f"observationdocument_recordeddtm:[{start_year}-{start_month} TO {end_year}-{end_month}]",
        )

    if len(current_pat_raw) == 0:

        features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        )

    features_data = current_pat_raw[
        current_pat_raw["obscatalogmasteritem_displayname"] == "CORE_RESUS_STATUS"
    ].copy()

    # screen and purge dud values
    # features_data =  features_data[(features_data['observation_valuetext_analysed'].astype(float)<20)& (features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    # features_data.dropna(inplace=True)

    # -----------------------------------------------------------------

    # features_data = current_pat_raw[current_pat_raw['obscatalogmasteritem_displayname']=='CORE_RESUS_STATUS'].copy()
    # features_data =  features_data[(features_data['observation_valuetext_analysed'].astype(float)<20)& (features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    # features_data.dropna(inplace=True)

    term = "CORE_RESUS_STATUS".lower()

    if len(features_data) > 0:
        features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        ).copy()
        features[f"{term}_For cardiopulmonary resuscitation"] = len(
            features_data[
                features_data["observation_valuetext_analysed"]
                == "For cardiopulmonary resuscitation"
            ]
        )
        features[f"{term}_Not for cardiopulmonary resuscitation"] = len(
            features_data[
                features_data["observation_valuetext_analysed"]
                == "Not for cardiopulmonary resuscitation"
            ]
        )
    else:
        features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        ).copy()
        features[f"{term}_For cardiopulmonary resuscitation"] = len(
            features_data[
                features_data["observation_valuetext_analysed"]
                == "For cardiopulmonary resuscitation"
            ]
        )
        features[f"{term}_Not for cardiopulmonary resuscitation"] = len(
            features_data[
                features_data["observation_valuetext_analysed"]
                == "Not for cardiopulmonary resuscitation"
            ]
        )

    return features

In [ ]:
def get_news(current_pat_client_id_code, target_date_range, pat_batch):

    start_year, start_month, end_year, end_month = get_start_end_year_month(
        target_date_range
    )

    if batch_mode:
        current_pat_raw_news = filter_dataframe_by_timestamp(
            pat_batch,
            start_year,
            start_month,
            end_year,
            end_month,
            "observationdocument_recordeddtm",
        )

    else:
        current_pat_raw_news = cohort_searcher_with_terms_and_search(
            index_name="observations",
            fields_list="""observation_guid client_idcode	obscatalogmasteritem_displayname	observation_valuetext_analysed	observationdocument_recordeddtm clientvisit_visitidcode""".split(),
            term_name="client_idcode.keyword",
            entered_list=[current_pat_client_id_code],
            search_string='obscatalogmasteritem_displayname:("NEWS" OR "NEWS2") AND '
            + f"observationdocument_recordeddtm:[{start_year}-{start_month} TO {end_year}-{end_month}]",
        )

    # if(len(current_pat_raw_news)==0):

    news_features = pd.DataFrame(
        data=[current_pat_client_id_code], columns=["client_idcode"]
    )

    news_features_data = current_pat_raw_news[
        current_pat_raw_news["obscatalogmasteritem_displayname"] == "NEWS2_Score"
    ].copy()

    # screen and purge dud values
    news_features_data = news_features_data[
        (news_features_data["observation_valuetext_analysed"].astype(float) < 20)
        & (news_features_data["observation_valuetext_analysed"].astype(float) > -20)
    ].copy()

    if len(news_features_data) > 0:
        news_features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        ).copy()
        value_array = (
            news_features_data["observation_valuetext_analysed"].dropna().astype(float)
        )
        news_features["news_score_mean"] = value_array.mean()
        news_features["news_score_median"] = value_array.median()
        news_features["news_score_std"] = value_array.std()
        news_features["news_score_max"] = max(value_array)
        news_features["news_score_min"] = min(value_array)
        news_features["news_score_n"] = value_array.shape[0]
    else:
        news_features["news_score_mean"] = np.nan
        news_features["news_score_median"] = np.nan
        news_features["news_score_std"] = np.nan
        news_features["news_score_max"] = np.nan
        news_features["news_score_min"] = np.nan
        news_features["news_score_n"] = np.nan

    # -----------------------------------------------------------------
    news_features_data = current_pat_raw_news[
        current_pat_raw_news["obscatalogmasteritem_displayname"] == "NEWS_Systolic_BP"
    ].copy()
    # news_features_data =  news_features_data[(news_features_data['observation_valuetext_analysed'].astype(float)<20)& (news_features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    news_features_data.dropna(subset=["observation_valuetext_analysed"], inplace=True)

    if len(news_features_data) > 0:
        # news_features = pd.DataFrame(data = [current_pat_client_id_code] , columns =['client_idcode']).copy()
        value_array = (
            news_features_data["observation_valuetext_analysed"]
            .dropna()
            .dropna()
            .astype(float)
        )
        news_features["news_systolic_bp_mean"] = value_array.mean()
        news_features["news_systolic_bp_median"] = value_array.median()
        news_features["news_systolic_bp_std"] = value_array.std()
        news_features["news_systolic_bp_max"] = max(value_array)
        news_features["news_systolic_bp_min"] = min(value_array)
        news_features["news_systolic_bp_n"] = value_array.shape[0]
    else:

        news_features["news_systolic_bp_mean"] = np.nan
        news_features["news_systolic_bp_median"] = np.nan
        news_features["news_systolic_bp_std"] = np.nan
        news_features["news_systolic_bp_max"] = np.nan
        news_features["news_systolic_bp_min"] = np.nan
        news_features["news_systolic_bp_n"] = np.nan

    # -----------------------------------------------------------------
    news_features_data = current_pat_raw_news[
        current_pat_raw_news["obscatalogmasteritem_displayname"] == "NEWS_Diastolic_BP"
    ].copy()
    # news_features_data =  news_features_data[(news_features_data['observation_valuetext_analysed'].astype(float)<20)& (news_features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    news_features_data.dropna(subset=["observation_valuetext_analysed"], inplace=True)

    if len(news_features_data) > 0:
        # news_features = pd.DataFrame(data = [current_pat_client_id_code] , columns =['client_idcode']).copy()
        value_array = (
            news_features_data["observation_valuetext_analysed"].dropna().astype(float)
        )
        news_features["news_diastolic_bp_mean"] = value_array.mean()
        news_features["news_diastolic_bp_median"] = value_array.median()
        news_features["news_diastolic_bp_std"] = value_array.std()
        news_features["news_diastolic_bp_max"] = max(value_array)
        news_features["news_diastolic_bp_min"] = min(value_array)
        news_features["news_diastolic_bp_n"] = value_array.shape[0]
    else:
        news_features["news_diastolic_bp_mean"] = np.nan
        news_features["news_diastolic_bp_median"] = np.nan
        news_features["news_diastolic_bp_std"] = np.nan
        news_features["news_diastolic_bp_max"] = np.nan
        news_features["news_diastolic_bp_min"] = np.nan
        news_features["news_diastolic_bp_n"] = np.nan

    # -----------------------------------------------------------------
    news_features_data = current_pat_raw_news[
        current_pat_raw_news["obscatalogmasteritem_displayname"]
        == "NEWS_Respiration_Rate"
    ].copy()
    # news_features_data =  news_features_data[(news_features_data['observation_valuetext_analysed'].astype(float)<20)& (news_features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    news_features_data.dropna(subset=["observation_valuetext_analysed"], inplace=True)

    if len(news_features_data) > 0:
        # news_features = pd.DataFrame(data = [current_pat_client_id_code] , columns =['client_idcode']).copy()
        value_array = (
            news_features_data["observation_valuetext_analysed"].dropna().astype(float)
        )
        news_features["news_respiration_rate_mean"] = value_array.mean()
        news_features["news_respiration_rate_median"] = value_array.median()
        news_features["news_respiration_rate_std"] = value_array.std()
        news_features["news_respiration_rate_max"] = max(value_array)
        news_features["news_respiration_rate_min"] = min(value_array)
        news_features["news_respiration_rate_n"] = value_array.shape[0]
    else:
        news_features["news_respiration_rate_mean"] = np.nan
        news_features["news_respiration_rate_median"] = np.nan
        news_features["news_respiration_rate_std"] = np.nan
        news_features["news_respiration_rate_max"] = np.nan
        news_features["news_respiration_rate_min"] = np.nan
        news_features["news_respiration_rate_n"] = np.nan

    # -----------------------------------------------------------------

    news_features_data = current_pat_raw_news[
        current_pat_raw_news["obscatalogmasteritem_displayname"] == "NEWS_Heart_Rate"
    ].copy()

    news_features_data.dropna(subset=["observation_valuetext_analysed"], inplace=True)

    if len(news_features_data) > 0:

        value_array = (
            news_features_data["observation_valuetext_analysed"].dropna().astype(float)
        )
        news_features["news_heart_rate_mean"] = value_array.mean()
        news_features["news_heart_rate_median"] = value_array.median()
        news_features["news_heart_rate_std"] = value_array.std()
        news_features["news_heart_rate_max"] = max(value_array)
        news_features["news_heart_rate_min"] = min(value_array)
        news_features["news_heart_rate_n"] = value_array.shape[0]
    else:
        news_features["news_heart_rate_mean"] = np.nan
        news_features["news_heart_rate_median"] = np.nan
        news_features["news_heart_rate_std"] = np.nan
        news_features["news_heart_rate_max"] = np.nan
        news_features["news_heart_rate_min"] = np.nan
        news_features["news_heart_rate_n"] = np.nan

    # -----------------------------------------------------------------

    news_features_data = current_pat_raw_news[
        current_pat_raw_news["obscatalogmasteritem_displayname"]
        == "NEWS_Oxygen_Saturation"
    ].copy()
    # news_features_data =  news_features_data[(news_features_data['observation_valuetext_analysed'].astype(float)<20)& (news_features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    news_features_data.dropna(subset=["observation_valuetext_analysed"], inplace=True)

    if len(news_features_data) > 0:

        value_array = (
            news_features_data["observation_valuetext_analysed"].dropna().astype(float)
        )
        news_features["news_oxygen_saturation_mean"] = value_array.mean()
        news_features["news_oxygen_saturation_median"] = value_array.median()
        news_features["news_oxygen_saturation_std"] = value_array.std()
        news_features["news_oxygen_saturation_max"] = max(value_array)
        news_features["news_oxygen_saturation_min"] = min(value_array)
        news_features["news_oxygen_saturation_n"] = value_array.shape[0]
    else:
        news_features["news_oxygen_saturation_mean"] = np.nan
        news_features["news_oxygen_saturation_median"] = np.nan
        news_features["news_oxygen_saturation_std"] = np.nan
        news_features["news_oxygen_saturation_max"] = np.nan
        news_features["news_oxygen_saturation_min"] = np.nan
        news_features["news_oxygen_saturation_n"] = np.nan

    # -----------------------------------------------------------------

    news_features_data = current_pat_raw_news[
        current_pat_raw_news["obscatalogmasteritem_displayname"] == "NEWS Temperature"
    ].copy()
    # news_features_data =  news_features_data[(news_features_data['observation_valuetext_analysed'].astype(float)<20)& (news_features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    news_features_data.dropna(subset=["observation_valuetext_analysed"], inplace=True)

    if len(news_features_data) > 0:
        # news_features = pd.DataFrame(data = [current_pat_client_id_code] , columns =['client_idcode']).copy()
        value_array = (
            news_features_data["observation_valuetext_analysed"].dropna().astype(float)
        )
        news_features["news_temperature_mean"] = value_array.mean()
        news_features["news_temperature_median"] = value_array.median()
        news_features["news_temperature_std"] = value_array.std()
        news_features["news_temperature_max"] = max(value_array)
        news_features["news_temperature_min"] = min(value_array)
        news_features["news_temperature_n"] = value_array.shape[0]
    else:
        news_features["news_temperature_mean"] = np.nan
        news_features["news_temperature_median"] = np.nan
        news_features["news_temperature_std"] = np.nan
        news_features["news_temperature_max"] = np.nan
        news_features["news_temperature_min"] = np.nan
        news_features["news_temperature_n"] = np.nan

    # -----------------------------------------------------------------

    news_features_data = current_pat_raw_news[
        current_pat_raw_news["obscatalogmasteritem_displayname"] == "NEWS_AVPU"
    ].copy()
    # news_features_data =  news_features_data[(news_features_data['observation_valuetext_analysed'].astype(float)<20)& (news_features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    news_features_data.dropna(subset=["observation_valuetext_analysed"], inplace=True)

    if len(news_features_data) > 0:
        # news_features = pd.DataFrame(data = [current_pat_client_id_code] , columns =['client_idcode']).copy()
        value_array = (
            news_features_data["observation_valuetext_analysed"].dropna().astype(float)
        )
        news_features["news_avpu_mean"] = value_array.mean()
        news_features["news_avpu_median"] = value_array.median()
        news_features["news_avpu_std"] = value_array.std()
        news_features["news_avpu_max"] = max(value_array)
        news_features["news_avpu_min"] = min(value_array)
        news_features["news_avpu_n"] = value_array.shape[0]
    else:
        news_features["news_avpu_mean"] = np.nan
        news_features["news_avpu_median"] = np.nan
        news_features["news_avpu_std"] = np.nan
        news_features["news_avpu_max"] = np.nan
        news_features["news_avpu_min"] = np.nan
        news_features["news_avpu_n"] = np.nan

    # -----------------------------------------------------------------

    news_features_data = current_pat_raw_news[
        current_pat_raw_news["obscatalogmasteritem_displayname"]
        == "NEWS_Supplemental_Oxygen"
    ].copy()
    # news_features_data =  news_features_data[(news_features_data['observation_valuetext_analysed'].astype(float)<20)& (news_features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    news_features_data.dropna(subset=["observation_valuetext_analysed"], inplace=True)

    term = "supplemental_oxygen"

    if len(news_features_data) > 0:
        # news_features = pd.DataFrame(data = [current_pat_client_id_code] , columns =['client_idcode']).copy()
        value_array = (
            news_features_data["observation_valuetext_analysed"].dropna().astype(float)
        )
        news_features[f"news_{term}_mean"] = value_array.mean()
        news_features[f"news_{term}_median"] = value_array.median()
        news_features[f"news_{term}_std"] = value_array.std()
        news_features[f"news_{term}_max"] = max(value_array)
        news_features[f"news_{term}_min"] = min(value_array)
        news_features[f"news_{term}_n"] = value_array.shape[0]
    else:
        news_features[f"news_{term}_mean"] = np.nan
        news_features[f"news_{term}_median"] = np.nan
        news_features[f"news_{term}_std"] = np.nan
        news_features[f"news_{term}_max"] = np.nan
        news_features[f"news_{term}_min"] = np.nan
        news_features[f"news_{term}_n"] = np.nan

    # -----------------------------------------------------------------

    news_features_data = current_pat_raw_news[
        current_pat_raw_news["obscatalogmasteritem_displayname"] == "NEWS2_Sp02_Target"
    ].copy()
    # news_features_data =  news_features_data[(news_features_data['observation_valuetext_analysed'].astype(float)<20)& (news_features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    news_features_data.dropna(subset=["observation_valuetext_analysed"], inplace=True)

    term = "Sp02_Target".lower()

    if len(news_features_data) > 0:
        # news_features = pd.DataFrame(data = [current_pat_client_id_code] , columns =['client_idcode']).copy()
        value_array = (
            news_features_data["observation_valuetext_analysed"].dropna().astype(float)
        )
        news_features[f"news_{term}_mean"] = value_array.mean()
        news_features[f"news_{term}_median"] = value_array.median()
        news_features[f"news_{term}_std"] = value_array.std()
        news_features[f"news_{term}_max"] = max(value_array)
        news_features[f"news_{term}_min"] = min(value_array)
        news_features[f"news_{term}_n"] = value_array.shape[0]
    else:
        news_features[f"news_{term}_mean"] = np.nan
        news_features[f"news_{term}_median"] = np.nan
        news_features[f"news_{term}_std"] = np.nan
        news_features[f"news_{term}_max"] = np.nan
        news_features[f"news_{term}_min"] = np.nan
        news_features[f"news_{term}_n"] = np.nan

    # -----------------------------------------------------------------

    news_features_data = current_pat_raw_news[
        current_pat_raw_news["obscatalogmasteritem_displayname"] == "NEWS2_Sp02_Scale"
    ].copy()
    # news_features_data =  news_features_data[(news_features_data['observation_valuetext_analysed'].astype(float)<20)& (news_features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    news_features_data.dropna(subset=["observation_valuetext_analysed"], inplace=True)

    term = "Sp02_Scale".lower()

    if len(news_features_data) > 0:
        # news_features = pd.DataFrame(data = [current_pat_client_id_code] , columns =['client_idcode']).copy()
        value_array = (
            news_features_data["observation_valuetext_analysed"].dropna().astype(float)
        )
        news_features[f"news_{term}_mean"] = value_array.mean()
        news_features[f"news_{term}_median"] = value_array.median()
        news_features[f"news_{term}_std"] = value_array.std()
        news_features[f"news_{term}_max"] = max(value_array)
        news_features[f"news_{term}_min"] = min(value_array)
        news_features[f"news_{term}_n"] = value_array.shape[0]
    else:
        news_features[f"news_{term}_mean"] = np.nan
        news_features[f"news_{term}_median"] = np.nan
        news_features[f"news_{term}_std"] = np.nan
        news_features[f"news_{term}_max"] = np.nan
        news_features[f"news_{term}_min"] = np.nan
        news_features[f"news_{term}_n"] = np.nan

    # -----------------------------------------------------------------

    news_features_data = current_pat_raw_news[
        current_pat_raw_news["obscatalogmasteritem_displayname"] == "NEWS_Pulse_Type"
    ].copy()
    # news_features_data =  news_features_data[(news_features_data['observation_valuetext_analysed'].astype(float)<20)& (news_features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    news_features_data.dropna(subset=["observation_valuetext_analysed"], inplace=True)

    term = "pulse_type".lower()

    if len(news_features_data) > 0:
        # news_features = pd.DataFrame(data = [current_pat_client_id_code] , columns =['client_idcode']).copy()
        value_array = (
            news_features_data["observation_valuetext_analysed"].dropna().astype(float)
        )
        news_features[f"news_{term}_mean"] = value_array.mean()
        news_features[f"news_{term}_median"] = value_array.median()
        news_features[f"news_{term}_std"] = value_array.std()
        news_features[f"news_{term}_max"] = max(value_array)
        news_features[f"news_{term}_min"] = min(value_array)
        news_features[f"news_{term}_n"] = value_array.shape[0]
    else:
        news_features[f"news_{term}_mean"] = np.nan
        news_features[f"news_{term}_median"] = np.nan
        news_features[f"news_{term}_std"] = np.nan
        news_features[f"news_{term}_max"] = np.nan
        news_features[f"news_{term}_min"] = np.nan
        news_features[f"news_{term}_n"] = np.nan

    # -----------------------------------------------------------------

    news_features_data = current_pat_raw_news[
        current_pat_raw_news["obscatalogmasteritem_displayname"] == "NEWS_Pain_Score"
    ].copy()
    # news_features_data =  news_features_data[(news_features_data['observation_valuetext_analysed'].astype(float)<20)& (news_features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    news_features_data.dropna(subset=["observation_valuetext_analysed"], inplace=True)

    term = "Pain_Score".lower()

    if len(news_features_data) > 0:
        # news_features = pd.DataFrame(data = [current_pat_client_id_code] , columns =['client_idcode']).copy()
        value_array = news_features_data["observation_valuetext_analysed"].astype(float)
        news_features[f"news_{term}_mean"] = value_array.mean()
        news_features[f"news_{term}_median"] = value_array.median()
        news_features[f"news_{term}_std"] = value_array.std()
        news_features[f"news_{term}_max"] = max(value_array)
        news_features[f"news_{term}_min"] = min(value_array)
        news_features[f"news_{term}_n"] = value_array.shape[0]
    else:
        news_features[f"news_{term}_mean"] = np.nan
        news_features[f"news_{term}_median"] = np.nan
        news_features[f"news_{term}_std"] = np.nan
        news_features[f"news_{term}_max"] = np.nan
        news_features[f"news_{term}_min"] = np.nan
        news_features[f"news_{term}_n"] = np.nan

    # -----------------------------------------------------------------

    news_features_data = current_pat_raw_news[
        current_pat_raw_news["obscatalogmasteritem_displayname"] == "NEWS Oxygen Litres"
    ].copy()
    # news_features_data =  news_features_data[(news_features_data['observation_valuetext_analysed'].astype(float)<20)& (news_features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    news_features_data.dropna(subset=["observation_valuetext_analysed"], inplace=True)

    term = "oxygen_litres".lower()

    if len(news_features_data) > 0:
        # news_features = pd.DataFrame(data = [current_pat_client_id_code] , columns =['client_idcode']).copy()
        value_array = (
            news_features_data["observation_valuetext_analysed"].dropna().astype(float)
        )
        news_features[f"news_{term}_mean"] = value_array.mean()
        news_features[f"news_{term}_median"] = value_array.median()
        news_features[f"news_{term}_std"] = value_array.std()
        news_features[f"news_{term}_max"] = max(value_array)
        news_features[f"news_{term}_min"] = min(value_array)
        news_features[f"news_{term}_n"] = value_array.shape[0]
    else:
        news_features[f"news_{term}_mean"] = np.nan
        news_features[f"news_{term}_median"] = np.nan
        news_features[f"news_{term}_std"] = np.nan
        news_features[f"news_{term}_max"] = np.nan
        news_features[f"news_{term}_min"] = np.nan
        news_features[f"news_{term}_n"] = np.nan

    # -----------------------------------------------------------------

    news_features_data = current_pat_raw_news[
        current_pat_raw_news["obscatalogmasteritem_displayname"]
        == "NEWS Oxygen Delivery"
    ].copy()
    # news_features_data =  news_features_data[(news_features_data['observation_valuetext_analysed'].astype(float)<20)& (news_features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    news_features_data.dropna(subset=["observation_valuetext_analysed"], inplace=True)

    term = "oxygen_delivery".lower()

    if len(news_features_data) > 0:
        # news_features = pd.DataFrame(data = [current_pat_client_id_code] , columns =['client_idcode']).copy()
        value_array = (
            news_features_data["observation_valuetext_analysed"].dropna().astype(float)
        )
        news_features[f"news_{term}_mean"] = value_array.mean()
        news_features[f"news_{term}_median"] = value_array.median()
        news_features[f"news_{term}_std"] = value_array.std()
        news_features[f"news_{term}_max"] = max(value_array)
        news_features[f"news_{term}_min"] = min(value_array)
        news_features[f"news_{term}_n"] = value_array.shape[0]
    else:
        news_features[f"news_{term}_mean"] = np.nan
        news_features[f"news_{term}_median"] = np.nan
        news_features[f"news_{term}_std"] = np.nan
        news_features[f"news_{term}_max"] = np.nan
        news_features[f"news_{term}_min"] = np.nan
        news_features[f"news_{term}_n"] = np.nan

    # -----------------------------------------------------------------

    news_features_data = current_pat_raw_news[
        current_pat_raw_news["obscatalogmasteritem_displayname"]
        == "NEWS Oxygen Delivery"
    ].copy()
    # news_features_data =  news_features_data[(news_features_data['observation_valuetext_analysed'].astype(float)<20)& (news_features_data['observation_valuetext_analysed'].astype(float)>-20)].copy()
    news_features_data.dropna(subset=["observation_valuetext_analysed"], inplace=True)

    term = "oxygen_delivery".lower()

    if len(news_features_data) > 0:
        # news_features = pd.DataFrame(data = [current_pat_client_id_code] , columns =['client_idcode']).copy()
        value_array = news_features_data["observation_valuetext_analysed"].astype(float)
        news_features[f"news_{term}_mean"] = value_array.mean()
        news_features[f"news_{term}_median"] = value_array.median()
        news_features[f"news_{term}_std"] = value_array.std()
        news_features[f"news_{term}_max"] = max(value_array)
        news_features[f"news_{term}_min"] = min(value_array)
        news_features[f"news_{term}_n"] = value_array.shape[0]
    else:
        news_features[f"news_{term}_mean"] = np.nan
        news_features[f"news_{term}_median"] = np.nan
        news_features[f"news_{term}_std"] = np.nan
        news_features[f"news_{term}_max"] = np.nan
        news_features[f"news_{term}_min"] = np.nan
        news_features[f"news_{term}_n"] = np.nan

    return news_features

In [ ]:
def get_bmi_features(current_pat_client_id_code, target_date_range, pat_batch):
    start_year, start_month, end_year, end_month = get_start_end_year_month(
        target_date_range
    )

    if batch_mode:
        current_pat_raw_bmi = filter_dataframe_by_timestamp(
            pat_batch,
            start_year,
            start_month,
            end_year,
            end_month,
            "observationdocument_recordeddtm",
        )

    else:
        current_pat_raw_bmi = cohort_searcher_with_terms_and_search(
            index_name="observations",
            fields_list="""observation_guid client_idcode	obscatalogmasteritem_displayname	observation_valuetext_analysed	observationdocument_recordeddtm clientvisit_visitidcode""".split(),
            term_name="client_idcode.keyword",
            entered_list=[current_pat_client_id_code],
            search_string='obscatalogmasteritem_displayname:("OBS BMI" OR "OBS Weight" OR "OBS height") AND '
            + f"observationdocument_recordeddtm:[{start_year}-{start_month} TO {end_year}-{end_month}]",
        )

    if (
        len(
            current_pat_raw_bmi[
                current_pat_raw_bmi["obscatalogmasteritem_displayname"]
                == "OBS BMI Calculation"
            ]
        )
    ) == 0:
        bmi_features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        )

    # get bmi features
    # set client id to current pat
    bmi_sample = current_pat_raw_bmi[
        current_pat_raw_bmi["obscatalogmasteritem_displayname"] == "OBS BMI Calculation"
    ]
    # screen and purge dud values
    bmi_sample = bmi_sample[
        (bmi_sample["observation_valuetext_analysed"].astype(float) < 200)
        & (bmi_sample["observation_valuetext_analysed"].astype(float) > 6)
    ]

    if len(bmi_sample) > 0:
        bmi_features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        ).copy()

        # if(len(current_pat_raw_bmi)>0):

        value_array = bmi_sample["observation_valuetext_analysed"].astype(float)

        bmi_features["bmi_mean"] = value_array.mean()
        bmi_features["bmi_median"] = value_array.median()
        bmi_features["bmi_std"] = value_array.std()
        bmi_features["bmi_high"] = int(bool(value_array.median() > 24.9))
        bmi_features["bmi_low"] = int(bool(value_array.median() < 18.5))
        bmi_features["bmi_extreme"] = int(bool(value_array.median() > 30))
        bmi_features["bmi_max"] = max(value_array)
        bmi_features["bmi_min"] = min(value_array)

    else:
        bmi_features = pd.DataFrame(
            data=[current_pat_client_id_code], columns=["client_idcode"]
        ).copy()
        bmi_features["bmi_mean"] = np.nan
        bmi_features["bmi_median"] = np.nan
        bmi_features["bmi_std"] = np.nan
        bmi_features["bmi_high"] = np.nan
        bmi_features["bmi_low"] = np.nan
        bmi_features["bmi_extreme"] = np.nan
        bmi_features["bmi_max"] = np.nan
        bmi_features["bmi_min"] = np.nan

    height_sample = current_pat_raw_bmi[
        current_pat_raw_bmi["obscatalogmasteritem_displayname"] == "OBS Height"
    ]
    # screen and purge dud values
    height_sample = height_sample[
        (height_sample["observation_valuetext_analysed"].astype(float) < 300)
        & (height_sample["observation_valuetext_analysed"].astype(float) > 30)
    ]

    # get height features
    if len(height_sample) > 0:

        if len(current_pat_raw_bmi) > 0:

            value_array = height_sample["observation_valuetext_analysed"].astype(float)

            bmi_features["height_mean"] = value_array.mean()
            bmi_features["height_median"] = value_array.median()
            bmi_features["height_std"] = value_array.std()

    else:
        bmi_features["height_mean"] = np.nan
        bmi_features["height_median"] = np.nan
        bmi_features["height_std"] = np.nan

    weight_sample = current_pat_raw_bmi[
        current_pat_raw_bmi["obscatalogmasteritem_displayname"] == "OBS Weight"
    ]
    # screen and purge dud values
    weight_sample = weight_sample[
        (weight_sample["observation_valuetext_analysed"].astype(float) < 800)
        & (weight_sample["observation_valuetext_analysed"].astype(float) > 1)
    ]

    # get weight features
    if len(weight_sample) > 0:

        if len(current_pat_raw_bmi) > 0:

            value_array = weight_sample["observation_valuetext_analysed"].astype(float)

            bmi_features["weight_mean"] = value_array.mean()
            bmi_features["weight_median"] = value_array.median()
            bmi_features["weight_std"] = value_array.std()
            bmi_features["weight_max"] = max(value_array)
            bmi_features["weight_min"] = min(value_array)

    else:
        bmi_features["weight_mean"] = np.nan
        bmi_features["weight_median"] = np.nan
        bmi_features["weight_std"] = np.nan
        bmi_features["weight_max"] = np.nan
        bmi_features["weight_min"] = np.nan

    return bmi_features

In [ ]:
current_pat_client_id_code = "%testclientidcode%"

In [ ]:
start_year = 1995
start_month = "1"
end_year = 2023
end_month = "11"

In [ ]:
cohort_searcher_with_terms_and_search(
    index_name="basic_observations",
    fields_list=[
        "client_idcode",
        "basicobs_itemname_analysed",
        "basicobs_value_numeric",
        "basicobs_entered",
        "clientvisit_serviceguid",
    ],
    term_name="client_idcode.keyword",
    entered_list=[current_pat_client_id_code],
    search_string="basicobs_value_numeric:* AND "
    + f"updatetime:[{start_year}-{start_month} TO {end_year}-{end_month}] ",
)  # In kibana can we pull the mean and std of each blood test for a reference.

In [ ]:
break

In [ ]:
def get_current_pat_bloods(current_pat_client_id_code, target_date_range, pat_batch):

    start_year, start_month, end_year, end_month = get_start_end_year_month(
        target_date_range
    )

    if batch_mode:
        current_pat_bloods = filter_dataframe_by_timestamp(
            pat_batch, start_year, start_month, end_year, end_month, "basicobs_entered"
        )

    else:
        current_pat_bloods = cohort_searcher_with_terms_and_search(
            index_name="basic_observations",
            fields_list=[
                "client_idcode",
                "basicobs_itemname_analysed",
                "basicobs_value_numeric",
                "basicobs_entered",
                "clientvisit_serviceguid",
            ],
            term_name="client_idcode.keyword",
            entered_list=[current_pat_client_id_code],
            search_string="basicobs_value_numeric:* AND "
            + f"updatetime:[{start_year}-{start_month} TO {end_year}-{end_month}] ",
        )  # In kibana can we pull the mean and std of each blood test for a reference.

    if batch_mode:
        current_pat_bloods["datetime"] = current_pat_bloods["basicobs_entered"].copy()

    else:
        current_pat_bloods["datetime"] = pd.Series(
            current_pat_bloods["basicobs_entered"]
        ).apply(convert_date)

    basicobs_itemname_analysed_list = list(
        current_pat_bloods["basicobs_itemname_analysed"].unique()
    )

    basicobs_itemname_analysed_df_dict = {
        elem: current_pat_bloods[current_pat_bloods.basicobs_itemname_analysed == elem]
        for elem in basicobs_itemname_analysed_list
    }

    df_unique = current_pat_bloods.copy()

    df_unique.drop_duplicates(subset="client_idcode", inplace=True)

    df_unique.reset_index(inplace=True)


    obs_columns_list = basicobs_itemname_analysed_list

    obs_columns_set = list(set(obs_columns_list))


    # print("Building obs_columns_set_columns_for_df")

    obs_columns_set_columns_for_df = []
    for i in range(0, len(obs_columns_set)):
        obs_columns_set_columns_for_df.append(obs_columns_set[i] + "_mean")
        obs_columns_set_columns_for_df.append(obs_columns_set[i] + "_median")
        obs_columns_set_columns_for_df.append(obs_columns_set[i] + "_mode")
        obs_columns_set_columns_for_df.append(obs_columns_set[i] + "_std")
        obs_columns_set_columns_for_df.append(obs_columns_set[i] + "_num-tests")
        obs_columns_set_columns_for_df.append(
            obs_columns_set[i] + "_days-since-last-test"
        )
        obs_columns_set_columns_for_df.append(obs_columns_set[i] + "_max")
        obs_columns_set_columns_for_df.append(obs_columns_set[i] + "_min")
        obs_columns_set_columns_for_df.append(obs_columns_set[i] + "_most-recent")
        obs_columns_set_columns_for_df.append(obs_columns_set[i] + "_earliest-test")
        obs_columns_set_columns_for_df.append(
            obs_columns_set[i] + "_days-between-first-last"
        )
        obs_columns_set_columns_for_df.append(
            obs_columns_set[i] + "_contains-extreme-low"
        )
        obs_columns_set_columns_for_df.append(
            obs_columns_set[i] + "_contains-extreme-high"
        )

    orig_columns = list(df_unique.columns)

    comb_cols = orig_columns + obs_columns_set_columns_for_df

    df_unique = df_unique.reindex(comb_cols, axis=1)

    df_unique = df_unique.copy()
    df_unique.drop(
        [
            "index",
            "_index",
            "_id",
            "_score",
            "basicobs_itemname_analysed",
            "basicobs_value_numeric",
            "basicobs_entered",
            "clientvisit_serviceguid",
            "datetime",
        ],
        inplace=True,
        axis=1,
    )

    if batch_mode:

        today = datetime.now(timezone.utc)

    else:
        today = datetime.today()


    df_unique_filtered = df_unique.copy()

    i = 0

    for j in range(0, len(basicobs_itemname_analysed_list)):
        col_name = basicobs_itemname_analysed_list[j]

        filtered_df = basicobs_itemname_analysed_df_dict.get(col_name)

        filtered_column_values = filtered_df.basicobs_value_numeric.astype(
            float
        )._get_numeric_data()

        df_len = len(filtered_df)
        # print(f"df_len {df_len}")
        if df_len >= 1:
            # Mean assurance*
            agg_val = float(filtered_column_values.values[0])

            df_unique_filtered.at[i, col_name + "_mean"] = agg_val

        if df_len >= 2:
            # try:
            # Mean
            agg_val = filtered_column_values.mean()

            df_unique_filtered.at[i, col_name + "_mean"] = agg_val

            # recent
            agg_val = filtered_df.sort_values(by="datetime").iloc[-1][
                "basicobs_value_numeric"
            ]

            df_unique_filtered.at[i, col_name + "_most-recent"] = agg_val

            # earliest-test
            agg_val = filtered_df.sort_values(by="datetime").iloc[0][
                "basicobs_value_numeric"
            ]
            df_unique_filtered.at[i, col_name + "_earliest-test"] = agg_val

            # days-since-last-test
            date_object = filtered_df.sort_values(by="datetime").iloc[-1]["datetime"]

            delta = today - date_object

            agg_val = delta.days

            df_unique_filtered.at[i, col_name + "_days-since-last-test"] = agg_val

            # n tests

            agg_val = len(filtered_column_values)

            df_unique_filtered.at[i, col_name + "_num-tests"] = agg_val

        if df_len >= 3:
            # try:
            # median
            # agg_val = np.median(filtered_df['basicobs_value_numeric'].to_numpy())
            agg_val = filtered_column_values.median()
            df_unique_filtered.at[i, col_name + "_median"] = agg_val

            # mode
            # agg_val = stats.mode(filtered_df['basicobs_value_numeric'].to_numpy(), keepdims=True)[0][0]

            agg_val = stats.mode(filtered_column_values)[0][0]
            df_unique_filtered.at[i, col_name + "_mode"] = agg_val

            # std
            agg_val = filtered_column_values.std()
            df_unique_filtered.at[i, col_name + "_std"] = agg_val

            # min
            agg_val = min(filtered_column_values)
            df_unique_filtered.at[i, col_name + "_min"] = agg_val

            # max
            agg_val = max(filtered_column_values)
            df_unique_filtered.at[i, col_name + "_max"] = agg_val

            # contains extreme low
            col_name_mean = (
                basicobs_itemname_analysed_df_dict.get(col_name)
                .basicobs_value_numeric._get_numeric_data()
                .mean()
            )
            col_name_std = (
                basicobs_itemname_analysed_df_dict.get(col_name)
                .basicobs_value_numeric._get_numeric_data()
                .std()
            )

            col_name_low = col_name_mean - (col_name_std * 3)

            agg_val = int(float(min(filtered_column_values)) < col_name_low)
            df_unique_filtered.at[i, col_name + "_contains-extreme-low"] = agg_val

            # contains extreme high
            col_name_high = col_name_mean + (col_name_std * 3)

            agg_val = int(float(max(filtered_column_values)) > col_name_high)

            df_unique_filtered.at[i, col_name + "_contains-extreme-high"] = agg_val

            # days_between earliest and last

            earliest = filtered_df.sort_values(by="datetime").iloc[-1]["datetime"]

            oldest = filtered_df.sort_values(by="datetime").iloc[-1]["datetime"]

            delta = earliest - oldest

            agg_val = delta.days

            df_unique_filtered.at[i, col_name + "_days-between-first-last"] = agg_val

            # current_pat_bloods = df_unique_filtered
    return df_unique_filtered

In [ ]:
def get_current_pat_drugs(current_pat_client_id_code, target_date_range, pat_batch):
    start_year, start_month, end_year, end_month = get_start_end_year_month(
        target_date_range
    )

    # Drugs
    if batch_mode:
        drugs = filter_dataframe_by_timestamp(
            pat_batch, start_year, start_month, end_year, end_month, "order_entered"
        )

    else:

        drugs = cohort_searcher_with_terms_and_search(
            index_name="order",
            fields_list="""client_idcode order_guid	order_name	order_summaryline order_holdreasontext	order_entered clientvisit_visitidcode""".split(),
            term_name="client_idcode.keyword",
            entered_list=[current_pat_client_id_code],
            search_string='order_typecode:"medication" AND '
            + f"updatetime:[{start_year}-{start_month} TO {end_year}-{end_month}] ",
        )

    current_pat_diagnostics = drugs.copy()

    if batch_mode:
        current_pat_diagnostics["datetime"] = current_pat_diagnostics[
            "order_entered"
        ].copy()

    else:
        current_pat_diagnostics["datetime"] = (
            pd.Series(current_pat_diagnostics["order_entered"])
            .dropna()
            .apply(convert_date)
        )

    order_name_list = list(current_pat_diagnostics["order_name"].unique())

    order_name_df_dict = {
        elem: current_pat_diagnostics[current_pat_diagnostics.order_name == elem]
        for elem in order_name_list
    }

    df_unique = current_pat_diagnostics.copy()

    df_unique.drop_duplicates(subset="client_idcode", inplace=True)

    df_unique.reset_index(inplace=True)


    obs_columns_list = order_name_list

    obs_columns_set = list(set(obs_columns_list))


    # print("Building obs_columns_set_columns_for_df")

    obs_columns_set_columns_for_df = []
    for i in range(0, len(obs_columns_set)):
        obs_columns_set_columns_for_df.append(obs_columns_set[i] + "_num-drug-order")
        obs_columns_set_columns_for_df.append(
            obs_columns_set[i] + "_days-since-last-drug-order"
        )
        obs_columns_set_columns_for_df.append(
            obs_columns_set[i] + "_days-between-first-last-drug"
        )

    orig_columns = list(df_unique.columns)

    comb_cols = orig_columns + obs_columns_set_columns_for_df

    df_unique = df_unique.reindex(comb_cols, axis=1)

    df_unique = df_unique.copy()
    df_unique.drop(
        [
            "_index",
            "_id",
            "_score",
            "order_guid",
            "order_name",
            "order_summaryline",
            "order_holdreasontext",
            "order_entered",
            "clientvisit_visitidcode",
        ],
        inplace=True,
        axis=1,
    )

    if batch_mode:

        today = datetime.now(timezone.utc)

    else:
        today = datetime.today()


    df_unique_filtered = df_unique.copy()

    i = 0

    for j in range(0, len(obs_columns_list)):
        col_name = obs_columns_list[j]

        filtered_df = order_name_df_dict.get(col_name)

        df_len = len(filtered_df)
        if df_len >= 1:
            # n tests

            agg_val = len(filtered_df)

            df_unique_filtered.at[i, col_name + "_num-drug-order"] = agg_val

            # days-since-last-test
            date_object = filtered_df.sort_values(by="datetime").iloc[-1]["datetime"]

            delta = today - date_object

            agg_val = delta.days

            df_unique_filtered.at[i, col_name + "_days-since-last-drug-order"] = agg_val

        if df_len >= 2:

            # days_between earliest and last

            earliest = filtered_df.sort_values(by="datetime").iloc[-1]["datetime"]

            oldest = filtered_df.sort_values(by="datetime").iloc[-1]["datetime"]

            delta = earliest - oldest

            agg_val = delta.days

            df_unique_filtered.at[i, col_name + "_days-between-first-last-drug"] = (
                agg_val
            )

    return df_unique_filtered

In [ ]:
def get_current_pat_diagnostics(
    current_pat_client_id_code, target_date_range, pat_batch
):

    start_year, start_month, end_year, end_month = get_start_end_year_month(
        target_date_range
    )

    # Diagnostic tests
    if batch_mode:
        diagnostics = filter_dataframe_by_timestamp(
            pat_batch, start_year, start_month, end_year, end_month, "order_entered"
        )

    else:
        diagnostics = cohort_searcher_with_terms_and_search(
            index_name="order",
            fields_list="""client_idcode order_guid	order_name	order_summaryline order_holdreasontext	order_entered clientvisit_visitidcode""".split(),
            term_name="client_idcode.keyword",
            entered_list=[current_pat_client_id_code],
            search_string='order_typecode:"diagnostic"AND '
            + f"updatetime:[{start_year}-{start_month} TO {end_year}-{end_month}] ",
        )

    current_pat_diagnostics = diagnostics.copy()

    if batch_mode:
        current_pat_diagnostics["datetime"] = current_pat_diagnostics[
            "order_entered"
        ].copy()

    else:
        current_pat_diagnostics["datetime"] = (
            pd.Series(current_pat_diagnostics["order_entered"])
            .dropna()
            .apply(convert_date)
        )

    order_name_list = list(current_pat_diagnostics["order_name"].unique())

    order_name_df_dict = {
        elem: current_pat_diagnostics[current_pat_diagnostics.order_name == elem]
        for elem in order_name_list
    }

    df_unique = current_pat_diagnostics.copy()

    df_unique.drop_duplicates(subset="client_idcode", inplace=True)

    df_unique.reset_index(inplace=True)


    obs_columns_list = order_name_list

    obs_columns_set = list(set(obs_columns_list))


    # print("Building obs_columns_set_columns_for_df")

    obs_columns_set_columns_for_df = []
    for i in range(0, len(obs_columns_set)):
        obs_columns_set_columns_for_df.append(
            obs_columns_set[i] + "_num-diagnostic-order"
        )
        obs_columns_set_columns_for_df.append(
            obs_columns_set[i] + "_days-since-last-diagnostic-order"
        )
        obs_columns_set_columns_for_df.append(
            obs_columns_set[i] + "_days-between-first-last-diagnostic"
        )

    orig_columns = list(df_unique.columns)

    comb_cols = orig_columns + obs_columns_set_columns_for_df

    df_unique = df_unique.reindex(comb_cols, axis=1)

    df_unique = df_unique.copy()
    df_unique.drop(
        [
            "_index",
            "_id",
            "_score",
            "order_guid",
            "order_name",
            "order_summaryline",
            "order_holdreasontext",
            "order_entered",
            "clientvisit_visitidcode",
        ],
        inplace=True,
        axis=1,
    )

    if batch_mode:

        today = datetime.now(timezone.utc)

    else:
        today = datetime.today()


    df_unique_filtered = df_unique.copy()

    i = 0

    for j in range(0, len(obs_columns_list)):
        col_name = obs_columns_list[j]

        filtered_df = order_name_df_dict.get(col_name)

        df_len = len(filtered_df)
        if df_len >= 1:
            # n tests

            agg_val = len(filtered_df)

            df_unique_filtered.at[i, col_name + "_num-diagnostic-order"] = agg_val

            # days-since-last-test
            date_object = filtered_df.sort_values(by="datetime").iloc[-1]["datetime"]

            delta = today - date_object

            agg_val = delta.days

            df_unique_filtered.at[i, col_name + "_days-since-last-diagnostic-order"] = (
                agg_val
            )

        if df_len >= 2:

            # days_between earliest and last

            earliest = filtered_df.sort_values(by="datetime").iloc[-1]["datetime"]

            oldest = filtered_df.sort_values(by="datetime").iloc[-1]["datetime"]

            delta = earliest - oldest

            agg_val = delta.days

            df_unique_filtered.at[
                i, col_name + "_days-between-first-last-diagnostic"
            ] = agg_val

    return df_unique_filtered

In [ ]:
def get_pat_batch_obs(current_pat_client_id_code, search_term):

    batch_target = cohort_searcher_with_terms_and_search(
        index_name="observations",
        fields_list="""observation_guid client_idcode	obscatalogmasteritem_displayname	observation_valuetext_analysed	observationdocument_recordeddtm clientvisit_visitidcode""".split(),
        term_name="client_idcode.keyword",
        entered_list=[current_pat_client_id_code],
        search_string=f'obscatalogmasteritem_displayname:("{search_term}") AND '
        + f"observationdocument_recordeddtm:[{global_start_year}-{global_start_month} TO {global_end_year}-{global_end_month}]",
    )

    return batch_target

In [ ]:
def get_pat_batch_news(current_pat_client_id_code, search_term):
    batch_target = cohort_searcher_with_terms_and_search(
        index_name="observations",
        fields_list="""observation_guid client_idcode	obscatalogmasteritem_displayname	observation_valuetext_analysed	observationdocument_recordeddtm clientvisit_visitidcode""".split(),
        term_name="client_idcode.keyword",
        entered_list=[current_pat_client_id_code],
        search_string='obscatalogmasteritem_displayname:("NEWS" OR "NEWS2") AND '
        + f"observationdocument_recordeddtm:[{global_start_year}-{global_start_month} TO {global_end_year}-{global_end_month}]",
    )

    return batch_target

In [ ]:
def get_pat_batch_bmi(current_pat_client_id_code, search_term):
    batch_target = cohort_searcher_with_terms_and_search(
        index_name="observations",
        fields_list="""observation_guid client_idcode	obscatalogmasteritem_displayname	observation_valuetext_analysed	observationdocument_recordeddtm clientvisit_visitidcode""".split(),
        term_name="client_idcode.keyword",
        entered_list=[current_pat_client_id_code],
        search_string='obscatalogmasteritem_displayname:("OBS BMI" OR "OBS Weight" OR "OBS height") AND '
        + f"observationdocument_recordeddtm:[{global_start_year}-{global_start_month} TO {global_end_year}-{global_end_month}]",
    )

    return batch_target

In [ ]:
def get_pat_batch_bloods(current_pat_client_id_code, search_term):
    batch_target = cohort_searcher_with_terms_and_search(
        index_name="basic_observations",
        fields_list=[
            "client_idcode",
            "basicobs_itemname_analysed",
            "basicobs_value_numeric",
            "basicobs_entered",
            "clientvisit_serviceguid",
        ],
        term_name="client_idcode.keyword",
        entered_list=[current_pat_client_id_code],
        search_string="basicobs_value_numeric:* AND "
        + f"updatetime:[{global_start_year}-{global_start_month} TO {global_end_year}-{global_end_month}] ",
    )  # In kibana can we pull the mean and std of each blood test for a reference.

    return batch_target

In [ ]:
def get_pat_batch_drugs(current_pat_client_id_code, search_term):
    batch_target = cohort_searcher_with_terms_and_search(
        index_name="order",
        fields_list="""client_idcode order_guid	order_name	order_summaryline order_holdreasontext	order_entered clientvisit_visitidcode""".split(),
        term_name="client_idcode.keyword",
        entered_list=[current_pat_client_id_code],
        search_string='order_typecode:"medication" AND '
        + f"updatetime:[{global_start_year}-{global_start_month} TO {global_end_year}-{global_end_month}] ",
    )

    return batch_target

In [ ]:
def get_pat_batch_diagnostics(current_pat_client_id_code, search_term):
    batch_target = cohort_searcher_with_terms_and_search(
        index_name="order",
        fields_list="""client_idcode order_guid	order_name	order_summaryline order_holdreasontext	order_entered clientvisit_visitidcode""".split(),
        term_name="client_idcode.keyword",
        entered_list=[current_pat_client_id_code],
        search_string='order_typecode:"diagnostic"AND '
        + f"updatetime:[{global_start_year}-{global_start_month} TO {global_end_year}-{global_end_month}] ",
    )

    return batch_target

In [ ]:
def get_pat_batch_epr_docs(current_pat_client_id_code, search_term):
    batch_target = cohort_searcher_with_terms_and_search(
        index_name="epr_documents",
        fields_list="""client_idcode document_guid	document_description	body_analysed updatetime clientvisit_visitidcode""".split(),
        term_name="client_idcode.keyword",
        entered_list=[current_pat_client_id_code],
        search_string=f"updatetime:[{global_start_year}-{global_start_month} TO {global_end_year}-{global_end_month}] ",
    )
    return batch_target

In [ ]:
def get_pat_batch_mct_docs(current_pat_client_id_code, search_term):
    batch_target = cohort_searcher_with_terms_and_search(
        index_name="observations",
        fields_list="""observation_guid client_idcode	obscatalogmasteritem_displayname	observation_valuetext_analysed	observationdocument_recordeddtm clientvisit_visitidcode""".split(),
        term_name="client_idcode.keyword",
        entered_list=[current_pat_client_id_code],
        search_string='obscatalogmasteritem_displayname:("AoMRC_ClinicalSummary_FT") AND '
        + f"observationdocument_recordeddtm:[{global_start_year}-{global_start_month} TO {global_end_year}-{global_end_month}]",
    )
    return batch_target

In [ ]:
def get_pat_batch_demo(current_pat_client_id_code, search_term):
    batch_target = cohort_searcher_with_terms_and_search(
        index_name="epr_documents",
        fields_list=[
            "client_idcode",
            "client_firstname",
            "client_lastname",
            "client_dob",
            "client_gendercode",
            "client_racecode",
            "client_deceaseddtm",
            "updatetime",
        ],
        term_name="client_idcode.keyword",
        entered_list=[current_pat_client_id_code],
        search_string=f"updatetime:[{global_start_year}-{global_start_month} TO {global_end_year}-{global_end_month}] ",
    )
    return batch_target

In [ ]:
# for k in tqdm(range(0,len(all_patient_list))):
# for k in tqdm(range(0,4)):
def main(current_pat_client_id_code, target_date_range):
    global skipped_counter
    global start_time
    start_time = time.time()

    if (
        current_pat_client_id_code + "_" + str(target_date_range)
        not in stripped_list_start
    ):

        stripped_list = [
            x.replace(".csv", "") for x in list_dir_wrapper(current_pat_lines_path)
        ]

        if (
            current_pat_client_id_code + "_" + str(target_date_range)
            not in stripped_list
        ):
            # print(start_time, current_pat_client_id_code)
            try:
                # current_pat_client_id_code = all_patient_list[k]

                patient_vector = []

                p_bar_entry = current_pat_client_id_code + "_" + str(target_date_range)
                #

                update_pbar(p_bar_entry, start_time, 0, "demo")

                if main_options.get("demo"):
                    current_pat_demo = get_demo(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(current_pat_demo)

                update_pbar(p_bar_entry, start_time, 1, "bmi")

                if main_options.get("bmi"):
                    bmi_features = get_bmi_features(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(bmi_features)

                update_pbar(p_bar_entry, start_time, 2, "bloods")

                if main_options.get("bloods"):
                    current_pat_bloods = get_current_pat_bloods(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(current_pat_bloods)

                update_pbar(p_bar_entry, start_time, 3, "drugs")

                if main_options.get("drugs"):
                    current_pat_drugs = get_current_pat_drugs(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(current_pat_drugs)

                update_pbar(p_bar_entry, start_time, 4, "diagnostics")

                if main_options.get("diagnostics"):
                    current_pat_diagnostics = get_current_pat_diagnostics(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(current_pat_diagnostics)

                # update_pbar(current_pat_client_id_code, start_time, 5, 'annotations', n_docs_to_annotate)

                if main_options.get("annotations"):
                    df_pat_target = get_current_pat_annotations(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(df_pat_target)

                if main_options.get("annotations_mrc"):
                    df_pat_target = get_current_pat_annotations_mrc_cs(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 1, "core_02")

                if main_options.get("core_02"):
                    df_pat_target = get_core_02(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 2, "bed")

                if main_options.get("bed"):
                    df_pat_target = get_bed(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 3, "vte_status")

                if main_options.get("vte_status"):
                    df_pat_target = get_vte_status(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 4, "hosp_site")

                if main_options.get("hosp_site"):
                    df_pat_target = get_hosp_site(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 1, "core_resus")

                if main_options.get("core_resus"):
                    df_pat_target = get_core_resus(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 2, "news")

                if main_options.get("news"):
                    df_pat_target = get_news(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 2, "concatenating")

                target_date_vector = enum_target_date_vector(
                    target_date_range, current_pat_client_id_code
                )

                patient_vector.append(target_date_vector)

                pat_concatted = pd.concat(patient_vector, axis=1)

                pat_concatted.drop("client_idcode", axis=1, inplace=True)

                pat_concatted.insert(0, "client_idcode", current_pat_client_id_code)

                update_pbar(p_bar_entry, start_time, 2, "saving...")

                output_path = (
                    current_pat_line_path
                    + current_pat_client_id_code
                    + "/"
                    + str(current_pat_client_id_code)
                    + "_"
                    + str(target_date_range)
                    + ".csv"
                )

                if not remote_dump:

                    pat_concatted.to_csv(output_path)
                else:

                    if multi_process:

                        write_remote(output_path, pat_concatted, stfp_obj)
                    else:
                        with sftp_client.open(output_path, "w") as file:
                            pat_concatted.to_csv(file)

                update_pbar(
                    p_bar_entry,
                    start_time,
                    2,
                    f"Done {len(pat_concatted.columns)} cols in {int(time.time() - start_time)}s, {int(len(pat_concatted.columns)/int(time.time() - start_time))} p/s",
                )

                # print(time.time() - start_time, current_pat_client_id_code)
            except RuntimeError:
                print("Caught runtime error... is torch?")
                print(RuntimeError)
                print("sleeping 1h")
                time.sleep(3600)

            except Exception as e:
                print(e)
                print(traceback.format_exc())
                print(f"Reproduce on {current_pat_client_id_code, target_date_range}")
                template = "An exception of type {0} occurred. Arguments:\n{1!r}"
                message = template.format(type(e).__name__, e.args)
                print(message)

        else:
            skipped_counter = skipped_counter + 1
            pass
            # print(f"{current_pat_client_id_code} done already")
            # pat_concatted

In [ ]:
# for k in tqdm(range(0,len(all_patient_list))):
# for k in tqdm(range(0,4)):
def main_multi(arg_list):

    current_pat_client_id_code, target_date_range = arg_list[0], arg_list[1]

    global skipped_counter
    global start_time
    start_time = time.time()

    if (
        current_pat_client_id_code + "_" + str(target_date_range)
        not in stripped_list_start
    ):

        stripped_list = [
            x.replace(".csv", "") for x in list_dir_wrapper(current_pat_lines_path)
        ]

        if (
            current_pat_client_id_code + "_" + str(target_date_range)
            not in stripped_list
        ):
            # print(start_time, current_pat_client_id_code)
            try:
                # current_pat_client_id_code = all_patient_list[k]

                patient_vector = []

                p_bar_entry = current_pat_client_id_code + "_" + str(target_date_range)
                #

                update_pbar(p_bar_entry, start_time, 0, "demo")

                if main_options.get("demo"):
                    current_pat_demo = get_demo(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(current_pat_demo)

                update_pbar(p_bar_entry, start_time, 1, "bmi")

                if main_options.get("bmi"):
                    bmi_features = get_bmi_features(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(bmi_features)

                update_pbar(p_bar_entry, start_time, 2, "bloods")

                if main_options.get("bloods"):
                    current_pat_bloods = get_current_pat_bloods(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(current_pat_bloods)

                update_pbar(p_bar_entry, start_time, 3, "drugs")

                if main_options.get("drugs"):
                    current_pat_drugs = get_current_pat_drugs(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(current_pat_drugs)

                update_pbar(p_bar_entry, start_time, 4, "diagnostics")

                if main_options.get("diagnostics"):
                    current_pat_diagnostics = get_current_pat_diagnostics(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(current_pat_diagnostics)

                # update_pbar(current_pat_client_id_code, start_time, 5, 'annotations', n_docs_to_annotate)

                if main_options.get("annotations"):
                    df_pat_target = get_current_pat_annotations(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(df_pat_target)

                if main_options.get("annotations_mrc"):
                    df_pat_target = get_current_pat_annotations_mrc_cs(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 1, "core_02")

                if main_options.get("core_02"):
                    df_pat_target = get_core_02(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 2, "bed")

                if main_options.get("bed"):
                    df_pat_target = get_bed(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 3, "vte_status")

                if main_options.get("vte_status"):
                    df_pat_target = get_vte_status(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 4, "hosp_site")

                if main_options.get("hosp_site"):
                    df_pat_target = get_hosp_site(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 1, "core_resus")

                if main_options.get("core_resus"):
                    df_pat_target = get_core_resus(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 2, "news")

                if main_options.get("news"):
                    df_pat_target = get_news(
                        current_pat_client_id_code, target_date_range
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 2, "concatenating")

                target_date_vector = enum_target_date_vector(
                    target_date_range, current_pat_client_id_code
                )

                patient_vector.append(target_date_vector)

                pat_concatted = pd.concat(patient_vector, axis=1)

                pat_concatted.drop("client_idcode", axis=1, inplace=True)

                pat_concatted.insert(0, "client_idcode", current_pat_client_id_code)

                update_pbar(p_bar_entry, start_time, 2, "saving...")

                if not remote_dump:

                    pat_concatted.to_csv(
                        current_pat_line_path
                        + str(current_pat_client_id_code)
                        + "_"
                        + str(target_date_range)
                        + ".csv"
                    )
                else:
                    with sftp_client.open(
                        current_pat_line_path
                        + str(current_pat_client_id_code)
                        + "_"
                        + str(target_date_range)
                        + ".csv",
                        "w",
                    ) as file:
                        pat_concatted.to_csv(file)

                update_pbar(
                    p_bar_entry,
                    start_time,
                    2,
                    f"Done {len(pat_concatted.columns)} cols in {int(time.time() - start_time)}s, {int(len(pat_concatted.columns)/int(time.time() - start_time))} p/s",
                )

                # print(time.time() - start_time, current_pat_client_id_code)
            except RuntimeError:
                print("Caught runtime error... is torch?")
                print(RuntimeError)
                print("sleeping 1h")
                time.sleep(3600)

            except Exception as e:
                print(e)
                print(traceback.format_exc())
                print(f"Reproduce on {current_pat_client_id_code, target_date_range}")
                template = "An exception of type {0} occurred. Arguments:\n{1!r}"
                message = template.format(type(e).__name__, e.args)
                print(message)

        else:
            skipped_counter = skipped_counter + 1
            pass
            # print(f"{current_pat_client_id_code} done already")
            # pat_concatted

In [ ]:
slow_execution_threshold_low = 10
slow_execution_threshold_high = 30
slow_execution_threshold_extreme = 60

In [ ]:
# problem

In [ ]:
current_pat_lines_path

In [ ]:
current_pat_line_path

In [ ]:
# list_dir_wrapper(current_pat_line_path, sftp_client)[0:5]

In [ ]:
# stripped_list[0:5]

In [ ]:
all_patient_list[0:5]

In [ ]:
len(stripped_list)

In [ ]:
# %%time
# if(remote_dump==False):
#     stripped_list = [x.replace(".csv","") for x in list_dir_wrapper(current_pat_line_path)]
#     all_patient_list = [fruit for fruit in all_patient_list if fruit not in stripped_list]

# else:
#     stripped_list = []

In [ ]:
len(stripped_list)

In [ ]:
priority_list_bool = False

if priority_list_bool:
    df_old_done = pd.read_csv(
        "/data/AS/Samora/HFE/HFE/v18/current_pat_lines_parts/current_pat_lines__part_0_merged.csv",
        usecols=["client_idcode", "Hemochromatosis (disorder)_count_subject_present"],
    )

    priority_list = df_old_done[
        df_old_done["Hemochromatosis (disorder)_count_subject_present"] > 0
    ]["client_idcode"].to_list()

    all_patient_list = priority_list  # + all_patient_list

In [ ]:
from tqdm import tqdm
from tqdm import trange

In [ ]:
pwd

In [ ]:
# medcat_model_pack_316666b47dfaac07.zip

In [ ]:
if aliencat:
    cat = CAT.load_model_pack(
        "/home/aliencat/samora/HFE/HFE/medcat_models/medcat_model_pack_316666b47dfaac07.zip"
    )
elif dgx:
    cat = CAT.load_model_pack(
        "/data/AS/Samora/HFE/HFE/v18/"
        + "medcat_models/20230328_trained_model_hfe_redone/medcat_model_pack_316666b47dfaac07"
    )

In [ ]:
# cat = CAT.load_model_pack('/data/AS/Samora/HFE/HFE/v18/' + 'medcat_models/20230328_trained_model_hfe_redone/medcat_model_pack_316666b47dfaac07');

In [ ]:
# cat = CAT.load_model_pack('/data/AS/Samora/HFE/HFE/v18/' + 'medcat_models/20230214_trained_model_hfe_redone/medcat_model_pack_bec3865f4a29ee20');

In [ ]:
random.seed()
random.shuffle(all_patient_list)

skipped_counter = 0
t = trange(
    len(all_patient_list),
    desc="Bar desc",
    leave=True,
    colour="GREEN",
    position=0,
    total=len(all_patient_list),
)

In [ ]:
combinations[0:3]

In [ ]:
all_patient_list_bu = all_patient_list.copy()

In [ ]:
# all_patient_list = all_patient_list[0:3]

In [ ]:
stripped_list = [
    x.replace(".csv", "") for x in list_dir_wrapper(current_pat_lines_path, sftp_client)
]

In [ ]:
len(stripped_list)

In [ ]:
%%time
if strip_list:
    stripped_list_start_copy = stripped_list.copy()

    container_list = []

    if remote_dump:
        ssh_client = paramiko.SSHClient()
        ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh_client.connect(hostname=hostname, username=username, password=password)

        sftp_client = ssh_client.open_sftp()

        for i in range(0, len(stripped_list)):
            try:
                if (
                    len(sftp_client.listdir(current_pat_lines_path + stripped_list[i]))
                    > 300
                ):
                    container_list.append(stripped_list[i])
            except:
                pass
    #             stripped_list_start_copy.remove(stripped_list_start[i])

    else:
        for i in tqdm(range(0, len(stripped_list))):
            if len(list_dir_wrapper(current_pat_lines_path + stripped_list[i])) > 300:
                container_list.append(stripped_list[i])

    stripped_list_start = container_list.copy()
    stripped_list = container_list.copy()
    if remote_dump:
        sftp_client.close()
        ssh_client.close()
else:
    stripped_list = []
    stripped_list_start = []

In [ ]:
all_arg_list = []
for i in range(0, len(all_patient_list)):
    for j in range(0, len(combinations)):
        all_arg_list.append([all_patient_list[i], combinations[j]])

In [ ]:
len(all_arg_list)

In [ ]:
# random.seed()
# random.shuffle(all_arg_list)

In [ ]:
# t = trange(len(all_arg_list), desc='Bar desc', leave=True, colour="GREEN", position=0, total=len(all_arg_list))
# skipped_counter = 0
# main_multi(all_arg_list[300])

In [ ]:
skip_additional_listdir = False

In [ ]:
def main_batch(
    current_pat_client_id_code,
    target_date_range,
    batch_demo=None,
    batch_smoking=None,
    batch_core_02=None,
    batch_bednumber=None,
    batch_vte=None,
    batch_hospsite=None,
    batch_resus=None,
    batch_news=None,
    batch_bmi=None,
    batch_diagnostics=None,
    batch_epr=None,
    batch_mct=None,
    batch_bloods=None,
    batch_drugs=None,
    sftp_obj=None,
):
    global skipped_counter
    global start_time
    start_time = time.time()

    already_done = False

    if current_pat_client_id_code not in stripped_list_start:

        if skip_additional_listdir:
            stripped_list = stripped_list_start
        else:

            if (
                len(
                    list_dir_wrapper(
                        current_pat_lines_path + current_pat_client_id_code, sftp_obj
                    )
                )
                >= 336
            ):
                already_done = True
                stripped_list_start.append(current_pat_client_id_code)
            stripped_list = stripped_list_start.copy()

            # stripped_list = []
        #             stripped_list = [x for x in list_dir_wrapper(current_pat_lines_path)]

        if current_pat_client_id_code not in stripped_list and not already_done:

            try:
                patient_vector = []

                p_bar_entry = current_pat_client_id_code + "_" + str(target_date_range)

                update_pbar(p_bar_entry, start_time, 0, "demo")

                if main_options.get("demo"):
                    current_pat_demo = get_demo(
                        current_pat_client_id_code, target_date_range, batch_demo
                    )
                    patient_vector.append(current_pat_demo)

                update_pbar(p_bar_entry, start_time, 1, "bmi")

                if main_options.get("bmi"):
                    bmi_features = get_bmi_features(
                        current_pat_client_id_code, target_date_range, batch_bmi
                    )
                    patient_vector.append(bmi_features)

                update_pbar(p_bar_entry, start_time, 2, "bloods")

                if main_options.get("bloods"):
                    current_pat_bloods = get_current_pat_bloods(
                        current_pat_client_id_code, target_date_range, batch_bloods
                    )
                    patient_vector.append(current_pat_bloods)

                update_pbar(p_bar_entry, start_time, 3, "drugs")

                if main_options.get("drugs"):
                    current_pat_drugs = get_current_pat_drugs(
                        current_pat_client_id_code, target_date_range, batch_drugs
                    )
                    patient_vector.append(current_pat_drugs)

                update_pbar(p_bar_entry, start_time, 4, "diagnostics")

                if main_options.get("diagnostics"):
                    current_pat_diagnostics = get_current_pat_diagnostics(
                        current_pat_client_id_code, target_date_range, batch_diagnostics
                    )
                    patient_vector.append(current_pat_diagnostics)

                if main_options.get("annotations"):
                    df_pat_target = get_current_pat_annotations(
                        current_pat_client_id_code,
                        target_date_range,
                        batch_epr,
                        sftp_obj,
                    )
                    patient_vector.append(df_pat_target)

                if main_options.get("annotations_mrc"):
                    df_pat_target = get_current_pat_annotations_mrc_cs(
                        current_pat_client_id_code,
                        target_date_range,
                        batch_mct,
                        sftp_obj,
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 1, "core_02")

                if main_options.get("core_02"):
                    df_pat_target = get_core_02(
                        current_pat_client_id_code, target_date_range, batch_core_02
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 2, "bed")

                if main_options.get("bed"):
                    df_pat_target = get_bed(
                        current_pat_client_id_code, target_date_range, batch_bednumber
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 3, "vte_status")

                if main_options.get("vte_status"):
                    df_pat_target = get_vte_status(
                        current_pat_client_id_code, target_date_range, batch_vte
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 4, "hosp_site")

                if main_options.get("hosp_site"):
                    df_pat_target = get_hosp_site(
                        current_pat_client_id_code, target_date_range, batch_hospsite
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 1, "core_resus")

                if main_options.get("core_resus"):
                    df_pat_target = get_core_resus(
                        current_pat_client_id_code, target_date_range, batch_resus
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 2, "news")

                if main_options.get("news"):
                    df_pat_target = get_news(
                        current_pat_client_id_code, target_date_range, batch_news
                    )
                    patient_vector.append(df_pat_target)

                update_pbar(p_bar_entry, start_time, 2, "concatenating")

                target_date_vector = enum_target_date_vector(
                    target_date_range, current_pat_client_id_code
                )

                patient_vector.append(target_date_vector)

                pat_concatted = pd.concat(patient_vector, axis=1)

                pat_concatted.drop("client_idcode", axis=1, inplace=True)

                pat_concatted.insert(0, "client_idcode", current_pat_client_id_code)

                update_pbar(p_bar_entry, start_time, 2, "saving...")

                output_path = (
                    current_pat_line_path
                    + current_pat_client_id_code
                    + "/"
                    + str(current_pat_client_id_code)
                    + "_"
                    + str(target_date_range)
                    + ".csv"
                )

                if not remote_dump:

                    pat_concatted.to_csv(output_path)
                else:

                    if multi_process:

                        write_remote(output_path, pat_concatted, sftp_obj)
                    else:
                        with sftp_client.open(output_path, "w") as file:
                            pat_concatted.to_csv(file)

                # display(type(pat_concatted))
                try:
                    update_pbar(
                        p_bar_entry,
                        start_time,
                        2,
                        f"Done {len(pat_concatted.columns)} cols in {int(time.time() - start_time)}s, {int((len(pat_concatted.columns)+1)/int(time.time() - start_time)+1)} p/s",
                    )
                except:
                    update_pbar(
                        p_bar_entry,
                        start_time,
                        2,
                        f"Columns n={len(pat_concatted.columns)}",
                    )
                    pass

                # display(pat_concatted)
                # print(time.time() - start_time, current_pat_client_id_code)
            except RuntimeError:
                print("Caught runtime error... is torch?")
                print(RuntimeError)
                print("sleeping 1h")
                time.sleep(3600)

            except Exception as e:
                print(e)
                print(traceback.format_exc())
                print(f"Reproduce on {current_pat_client_id_code, target_date_range}")
                template = "An exception of type {0} occurred. Arguments:\n{1!r}"
                message = template.format(type(e).__name__, e.args)
                print(message)

        else:
            if not multi_process:
                skipped_counter = skipped_counter + 1
            else:
                with skipped_counter.get_lock():
                    skipped_counter.value += 1
            pass
            # print(f"{current_pat_client_id_code} done already")
            # pat_concatted

In [ ]:
global_start_year, global_start_month, global_end_year, global_end_month = (
    "1995",
    "01",
    "2023",
    "11",
)

#get_pat batches

search_term = 'CORE_SmokingStatus'

batch_smoking = get_pat_batch_obs(current_pat_client_id_code, search_term)


search_term = 'CORE_SpO2'

batch_core_02 = get_pat_batch_obs(current_pat_client_id_code, search_term)


search_term = 'CORE_BedNumber3'

batch_bednumber = get_pat_batch_obs(current_pat_client_id_code, search_term)


search_term = 'CORE_VTE_STATUS'

batch_vte = get_pat_batch_obs(current_pat_client_id_code, search_term)


search_term = 'CORE_HospitalSite'

batch_hospsite = get_pat_batch_obs(current_pat_client_id_code, search_term)


search_term = 'CORE_RESUS_STATUS'

batch_resus = get_pat_batch_obs(current_pat_client_id_code, search_term)


search_term = None # inside function
batch_news = get_pat_batch_news(current_pat_client_id_code, search_term)


search_term = None # inside function
batch_bmi = get_pat_batch_bmi(current_pat_client_id_code, search_term)


search_term = None # inside function
batch_diagnostics = get_pat_batch_diagnostics(current_pat_client_id_code, search_term)

search_term = None # inside function
batch_drugs = get_pat_batch_drugs(current_pat_client_id_code, search_term)


search_term = None # inside function
batch_epr = get_pat_batch_epr_docs(current_pat_client_id_code, search_term)


search_term = None # inside function
batch_mct = get_pat_batch_mct_docs(current_pat_client_id_code, search_term)

search_term = None # inside function
batch_demo = get_pat_batch_demo(current_pat_client_id_code, search_term)

search_term = None # inside function
batch_bloods =  get_pat_batch_bloods(current_pat_client_id_code, search_term)

batch_bloods


target_date_range = (2022, 10)

target_date_range

#pd.set_option('display.max_columns', 500)

#%%prun


main_batch(current_pat_client_id_code,
               target_date_range,
               batch_demo = batch_demo,
               batch_smoking = batch_smoking,
               batch_core_02 = batch_core_02,
               batch_bednumber = batch_bednumber,
               batch_vte = batch_vte,
               batch_hospsite = batch_hospsite,
               batch_resus = batch_resus,
               batch_news = batch_news,
               batch_bmi = batch_bmi,
               batch_diagnostics = batch_diagnostics,
               batch_epr = batch_epr,
               batch_mct = batch_mct,
               batch_bloods = batch_bloods,
               batch_drugs = batch_drugs
            
              )


In [ ]:
annotate_only = False

if annotate_only:
    random.seed()
    random.shuffle(all_patient_list)

    skipped_counter = 0
    t = trange(
        len(all_patient_list),
        desc="Bar desc",
        leave=True,
        colour="GREEN",
        position=0,
        total=len(all_patient_list),
    )

    for i in t:
        global_start_year, global_start_month, global_end_year, global_end_month = (
            "1995",
            "01",
            "2023",
            "11",
        )

        current_pat_doc_batch = cohort_searcher_with_terms_and_search(
            index_name="epr_documents",
            fields_list="""client_idcode document_guid	document_description	body_analysed updatetime clientvisit_visitidcode""".split(),
            term_name="client_idcode.keyword",
            entered_list=[all_patient_list[i]],
            search_string=f"updatetime:[{global_start_year}-{global_start_month} TO {global_end_year}-{global_end_month}] ",
        )

        current_pat_doc_mct_batch = cohort_searcher_with_terms_and_search(
            index_name="observations",
            fields_list="""observation_guid client_idcode	obscatalogmasteritem_displayname	observation_valuetext_analysed	observationdocument_recordeddtm clientvisit_visitidcode""".split(),
            term_name="client_idcode.keyword",
            entered_list=[all_patient_list[i]],
            search_string='obscatalogmasteritem_displayname:("AoMRC_ClinicalSummary_FT") AND '
            + f"observationdocument_recordeddtm:[{global_start_year}-{global_start_month} TO {global_end_year}-{global_end_month}]",
        )

        for j in range(0, len(combinations)):
            try:
                if all_patient_list[i] not in stripped_list:
                    get_current_pat_annotations_batch_to_file(
                        all_patient_list[i], combinations[j], current_pat_doc_batch
                    )

                    get_current_pat_annotations_mct_batch_to_file(
                        all_patient_list[i], combinations[j], current_pat_doc_mct_batch
                    )
            except Exception as e:
                print(e)
                print(all_patient_list[i], combinations[j])

In [ ]:
current_pat_line_path

In [ ]:
def write_remote(path, csv_file, sftp_obj=None):
    # print("writing remote")
    if not share_sftp:
        ssh_client = paramiko.SSHClient()
        ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh_client.connect(hostname=hostname, username=username, password=password)

        sftp_client = ssh_client.open_sftp()
        sftp_obj = sftp_client

    with sftp_obj.open(path, "w") as file:
        csv_file.to_csv(file)

    if not share_sftp:
        sftp_obj.close()
        sftp_obj.close()

In [ ]:
random.seed()
random.shuffle(all_patient_list)

skipped_counter = 0
t = trange(
    len(all_patient_list),
    desc="Bar desc",
    leave=True,
    colour="GREEN",
    position=0,
    total=len(all_patient_list),
)

In [ ]:
# break

In [ ]:
def pat_maker(i):
    global skipped_counter
    global stripped_list

    current_pat_client_id_code = all_patient_list[i]

    p_bar_entry = current_pat_client_id_code

    start_time = time.time()

    update_pbar(p_bar_entry, start_time, 0, f"Pat_maker called on {i}...")

    # time.sleep(random.randint(1, 50))
    # i, sftp_obj = i[0], i[1]
    if remote_dump:
        ssh_client = paramiko.SSHClient()
        ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh_client.connect(
            hostname=hostname, username=username, password=password, timeout=60
        )

        sftp_obj = ssh_client.open_sftp()
    else:
        sftp_obj = None

    # get_pat batches

    stripped_list = stripped_list_start.copy()

    if current_pat_client_id_code not in stripped_list_start:

        update_pbar(p_bar_entry, start_time, 0, "Getting batches...")

        search_term = None  # inside function
        batch_epr = get_pat_batch_epr_docs(current_pat_client_id_code, search_term)

        search_term = None  # inside function
        batch_mct = get_pat_batch_mct_docs(current_pat_client_id_code, search_term)

        if not annot_first:

            search_term = "CORE_SmokingStatus"

            batch_smoking = get_pat_batch_obs(current_pat_client_id_code, search_term)

            search_term = "CORE_SpO2"

            batch_core_02 = get_pat_batch_obs(current_pat_client_id_code, search_term)

            search_term = "CORE_BedNumber3"

            batch_bednumber = get_pat_batch_obs(current_pat_client_id_code, search_term)

            search_term = "CORE_VTE_STATUS"

            batch_vte = get_pat_batch_obs(current_pat_client_id_code, search_term)

            search_term = "CORE_HospitalSite"

            batch_hospsite = get_pat_batch_obs(current_pat_client_id_code, search_term)

            search_term = "CORE_RESUS_STATUS"

            batch_resus = get_pat_batch_obs(current_pat_client_id_code, search_term)

            search_term = None  # inside function
            batch_news = get_pat_batch_news(current_pat_client_id_code, search_term)

            search_term = None  # inside function
            batch_bmi = get_pat_batch_bmi(current_pat_client_id_code, search_term)

            search_term = None  # inside function
            batch_diagnostics = get_pat_batch_diagnostics(
                current_pat_client_id_code, search_term
            )

            search_term = None  # inside function
            batch_drugs = get_pat_batch_drugs(current_pat_client_id_code, search_term)

            search_term = None  # inside function
            batch_demo = get_pat_batch_demo(current_pat_client_id_code, search_term)

            search_term = None  # inside function
            batch_bloods = get_pat_batch_bloods(current_pat_client_id_code, search_term)

        update_pbar(
            p_bar_entry, start_time, 0, f"Done batches in {time.time()-start_time}"
        )

        run_on_pat = False

        only_check_last = True

        last_check = all_patient_list[i] not in stripped_list

        skip_check = last_check

        for j in range(0, len(combinations)):
            try:
                if only_check_last:
                    run_on_pat = last_check
                else:
                    run_on_pat = all_patient_list[i] not in stripped_list

                if run_on_pat:
                    if annot_first:

                        get_current_pat_annotations_batch_to_file(
                            all_patient_list[i],
                            combinations[j],
                            batch_epr,
                            sftp_obj,
                            skip_check=skip_check,
                        )

                        get_current_pat_annotations_mct_batch_to_file(
                            all_patient_list[i],
                            combinations[j],
                            batch_mct,
                            sftp_obj,
                            skip_check=skip_check,
                        )

                    else:
                        main_batch(
                            all_patient_list[i],
                            combinations[j],
                            batch_demo=batch_demo,
                            batch_smoking=batch_smoking,
                            batch_core_02=batch_core_02,
                            batch_bednumber=batch_bednumber,
                            batch_vte=batch_vte,
                            batch_hospsite=batch_hospsite,
                            batch_resus=batch_resus,
                            batch_news=batch_news,
                            batch_bmi=batch_bmi,
                            batch_diagnostics=batch_diagnostics,
                            batch_epr=batch_epr,
                            batch_mct=batch_mct,
                            batch_bloods=batch_bloods,
                            batch_drugs=batch_drugs,
                            sftp_obj=sftp_obj,
                        )

            except Exception as e:
                print(e)
                print(
                    f"Exception in patmaker on {all_patient_list[i], combinations[j]}"
                )
                print(traceback.format_exc())
        if remote_dump:
            sftp_obj.close()
            ssh_client.close()
    else:
        if not multi_process:
            skipped_counter = skipped_counter + 1
            update_pbar(str(i), start_time, 0, f"Skipped {i}")
        else:
            with skipped_counter.get_lock():
                skipped_counter.value += 1
            update_pbar(str(i), start_time, 0, f"Skipped {i}")

In [ ]:
random.seed()
random.shuffle(all_patient_list)
all_patient_list = [x for x in all_patient_list if x not in stripped_list_start]

skipped_counter = 0
t = trange(
    len(all_patient_list),
    desc="Bar desc",
    leave=True,
    colour="GREEN",
    position=0,
    total=len(all_patient_list),
)

all_patient_list = [x for x in all_patient_list if x not in stripped_list_start]

skipped_counter = 0
t = trange(
    len(all_patient_list),
    desc="Bar desc",
    leave=True,
    colour="GREEN",
    position=0,
    total=len(all_patient_list),
)

In [ ]:
# pat_maker(12)

In [ ]:
# random.seed()
# random.shuffle(all_patient_list)

# skipped_counter = 0
# t = trange(10, desc='Bar desc', leave=True, colour="GREEN", position=0, total=10)

In [ ]:
# pat_maker(0)

In [ ]:
# ssh_client = paramiko.SSHClient()
# ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
# ssh_client.connect(hostname=hostname, username=username, password=password, timeout=10)

# sftp_client = ssh_client.open_sftp()

In [ ]:
# could annotate every document in the batch and store result in a dictionary of dates, use dict lookup to fill files in loop

In [ ]:
from multiprocessing import Value

skipped_counter = None


def init(args):
    """store the counter for later use"""
    global skipped_counter
    skipped_counter = args

In [ ]:
# init?
stripped_list = stripped_list_start.copy()

In [ ]:
multi_process = False

In [ ]:
%%time
# %%prun
pat_maker(0)
stripped_list_start.append(all_patient_list[0])

In [ ]:
skipped_counter = 0
t = trange(
    len(all_patient_list),
    desc="Bar desc",
    leave=True,
    colour="GREEN",
    position=0,
    total=len(all_patient_list),
)

In [ ]:
multi_process = True

# if(multi_process):
#     #sftp_client.close()
#     ssh_client.close()

if multi_process:

    if __name__ == "__main__":
        skipped_counter = Value("i", 0)

        pool = Pool(processes=3, initializer=init, initargs=(skipped_counter,))
        # pool = eventlet.GreenPool(size=1000)
        for patient in pool.imap(pat_maker, [x for x in range(len(all_patient_list))]):

            _
        pool.close()
else:
    for i in t:
        pat_maker(i)

# Threadpool

In [ ]:
from multiprocessing.dummy import Pool as ThreadPool

if multi_process:
    if __name__ == "__main__":
        skipped_counter = Value("i", 0)

        pool = ThreadPool(processes=8, initializer=init, initargs=(skipped_counter,))
        # pool = ThreadPool(80)
        results = pool.map(pat_maker, [x for x in range(0, len(all_patient_list))])
    pool.close()

In [ ]:
break